## 라이브러리등

In [1]:
pip install gspread pandas matplotlib plotly ipywidgets oauth2client

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
   ---------------------------------------- 57.6/57.6 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/98.2 kB ? eta -:--:--
   ---------------------------------------- 98.2/98.2 kB 1.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/209.8 kB ? eta -:--:--
   ---------------------------------------  204.8/209.8 kB 4.1 MB/s eta 0:00:01
   ---------------------------------------- 209.8/209.8 kB 4.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/96.9 kB ? eta -:--:--
   ---------------------------------------- 96.9/96.9 kB ? eta 0:00:00
   ---------------------------------------- 0.0/151.7 kB ? eta -:--:--
   ---------------------------------------- 151.7/151.7 kB 8.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [7]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/12.7 MB ? eta -:--:--
   ---------------------------------------- 0.1/12.7 MB 3.2 MB/s eta 0:00:04
   - -------------------------------------- 0.6/12.7 MB 9.6 MB/s eta 0:00:02
   --- ------------------------------------ 1.2/12.7 MB 12.6 MB/s eta 0:00:01
   ------ --------------------------------- 1.9/12.7 MB 13.7 MB/s eta 0:00:01
   ------- -------------------------------- 2.5/12.7 MB 14.3 MB/s eta 0:00:01
   --------- ------------------------------ 2.9/12.7 MB 14.2 MB/s eta 0:00:01
   ----------- ---------------------------- 3.7/12.7 MB 14.7 MB/s eta 0:00:01
   ------------ --------------------------- 4.1/12.7 MB 13.8 MB/s eta 0:00:01
   --------------- ------------------------ 4.9/12.7 MB 14.3 MB/s eta 0:00:01
   ----------------- ---------------------- 5.6/12.7 MB 14.9 MB/s eta 0:00:01
   ------------------- -------------------- 6.2/12.7 MB 14.6 MB/s eta 0:00

In [11]:
pip show google-api-python-client

Name: google-api-python-client
Version: 2.156.0
Summary: Google API Client Library for Python
Home-page: https://github.com/googleapis/google-api-python-client/
Author: Google LLC
Author-email: googleapis-packages@google.com
License: Apache 2.0
Location: C:\Users\User\AppData\Roaming\Python\Python312\site-packages
Requires: google-api-core, google-auth, google-auth-httplib2, httplib2, uritemplate
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [17]:
!python -m venv myenv
!myenv\Scripts\activate      # Windows
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

Defaulting to user installation because normal site-packages is not writeable


In [21]:
pip install PyDrive2

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/48.0 kB ? eta -:--:--
   -------- ------------------------------- 10.2/48.0 kB ? eta -:--:--
   ---------------------------------- ----- 41.0/48.0 kB 653.6 kB/s eta 0:00:01
   ---------------------------------------- 48.0/48.0 kB 598.9 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [27]:
import os
import io
import pandas as pd
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google.oauth2.service_account import Credentials

# 1. Google Drive 인증 설정
credentials_file = "credentials.json"  # credentials.json 파일 경로
SCOPES = ['https://www.googleapis.com/auth/drive/Key/']
creds = Credentials.from_service_account_file(credentials_file, scopes=SCOPES)

# Google Drive API 클라이언트 생성
drive_service = build('drive', 'v3', credentials=creds)

# 2. Google Drive 파일 목록 가져오기
def list_drive_files():
    results = drive_service.files().list(
        pageSize=100, fields="nextPageToken, files(id, name)"
    ).execute()
    items = results.get('files', [])
   
    if not items:
        print("Google Drive에 파일이 없습니다.")
        return []
   
    print("Google Drive 파일 목록:")
    for item in items:
        print(f"{item['name']} (ID: {item['id']})")
    return items

# 3. Google Drive 파일 다운로드
def download_file(file_id, save_path):
    request = drive_service.files().get_media(fileId=file_id)
    with io.FileIO(save_path, 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"다운로드 진행률: {int(status.progress() * 100)}%")
    print(f"파일 다운로드 완료: {save_path}")

# 4. 데이터 분석 (Pandas)
def analyze_csv(file_path):
    try:
        df = pd.read_csv(file_path, encoding='cp949')  # 파일 인코딩에 따라 cp949 또는 utf-8 사용
        print(f"CSV 파일 로드 완료: {file_path}")
        print("데이터 미리보기:")
        print(df.head())
       
        # 분석 예시: 열 이름, 데이터 통계
        print("\n열 이름:", df.columns.tolist())
        print("\n데이터 통계:")
        print(df.describe())
       
        return df
    except Exception as e:
        print(f"CSV 파일 분석 중 오류 발생: {e}")
        return None

# 5. 전체 프로세스 실행
def main():
    # Step 1: Google Drive에서 파일 목록 가져오기
    drive_files = list_drive_files()
   
    # Step 2: 특정 파일 선택 (여기서는 첫 번째 파일을 선택)
    if drive_files:
        target_file = drive_files[0]  # 첫 번째 파일 선택
        file_id = target_file['id']
        file_name = target_file['name']
       
        # Step 3: 파일 다운로드
        download_path = os.path.join(os.getcwd(), file_name)  # 현재 디렉토리에 저장
        download_file(file_id, download_path)
       
        # Step 4: 다운로드한 파일 분석
        if file_name.endswith('.csv'):
            analyze_csv(download_path)
        else:
            print(f"다운로드한 파일은 CSV 형식이 아닙니다: {file_name}")

# 실행
if __name__ == "__main__":
    main()

FileNotFoundError: [Errno 2] No such file or directory: 'credentials.json'

In [29]:
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google.oauth2.service_account import Credentials
import pandas as pd
import io
import os

# 1. 인증 파일 업로드 받기
def upload_credentials():
    from google.colab import files
    uploaded = files.upload()  # 인증 파일 업로드
    if 'credentials.json' not in uploaded:
        raise FileNotFoundError("업로드된 파일에 'credentials.json'이 없습니다.")
    print("인증 파일 업로드 완료: credentials.json")

# 2. Google Drive 인증
def authenticate_google_drive():
    SCOPES = ['https://www.googleapis.com/auth/drive']
    creds = Credentials.from_service_account_file("credentials.json", scopes=SCOPES)
    drive_service = build('drive', 'v3', credentials=creds)
    return drive_service

# 3. Google Drive 파일 목록 가져오기
def list_drive_files(drive_service):
    results = drive_service.files().list(
        pageSize=10, fields="nextPageToken, files(id, name)"
    ).execute()
    items = results.get('files', [])
    if not items:
        print("Google Drive에 파일이 없습니다.")
    else:
        print("Google Drive 파일 목록:")
        for item in items:
            print(f"{item['name']} (ID: {item['id']})")
    return items

# 4. Google Drive 파일 다운로드
def download_file(drive_service, file_id, file_name):
    request = drive_service.files().get_media(fileId=file_id)
    with io.FileIO(file_name, 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"다운로드 진행률: {int(status.progress() * 100)}%")
    print(f"파일 다운로드 완료: {file_name}")

# 5. 데이터 분석 (Pandas)
def analyze_csv(file_path):
    try:
        df = pd.read_csv(file_path, encoding='utf-8')  # 필요한 경우 'cp949'로 변경
        print(f"CSV 파일 로드 완료: {file_path}")
        print("데이터 미리보기:")
        print(df.head())

        # 간단한 분석: 열 이름과 데이터 통계
        print("\n열 이름:", df.columns.tolist())
        print("\n데이터 통계:")
        print(df.describe())

        return df
    except Exception as e:
        print(f"CSV 파일 분석 중 오류 발생: {e}")
        return None

# 6. 전체 실행
def main():
    # Step 1: 인증 파일 업로드
    upload_credentials()

    # Step 2: Google Drive 인증
    drive_service = authenticate_google_drive()

    # Step 3: Google Drive 파일 목록 가져오기
    drive_files = list_drive_files(drive_service)

    # Step 4: 특정 파일 다운로드 및 분석 (첫 번째 파일 예시)
    if drive_files:
        target_file = drive_files[0]  # 첫 번째 파일 선택
        file_id = target_file['id']
        file_name = target_file['name']

        # 파일 다운로드
        download_file(drive_service, file_id, file_name)

        # 다운로드한 파일 분석
        if file_name.endswith('.csv'):
            analyze_csv(file_name)
        else:
            print(f"다운로드한 파일은 CSV 형식이 아닙니다: {file_name}")

# 실행
if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'google.colab'

In [33]:
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google.oauth2.service_account import Credentials
import pandas as pd
import io
import os


# 1. 인증 파일 경로 설정
def set_credentials():
    credentials_path = "D:\\구글key\\credentials (1).json"  # 실제 credentials.json 파일 경로 입력
    if not os.path.exists(credentials_path):
        raise FileNotFoundError("credentials.json 파일을 찾을 수 없습니다. 경로를 확인하세요.")
    print("인증 파일 경로 확인 완료: credentials.json")
    return credentials_path

# 2. Google Drive 인증
def authenticate_google_drive(credentials_path):
    SCOPES = ['https://www.googleapis.com/auth/drive']
    creds = Credentials.from_service_account_file(credentials_path, scopes=SCOPES)
    drive_service = build('drive', 'v3', credentials=creds)
    return drive_service

# 3. Google Drive 파일 목록 가져오기
def list_drive_files(drive_service):
    results = drive_service.files().list(
        pageSize=10, fields="nextPageToken, files(id, name)"
    ).execute()
    items = results.get('files', [])
    if not items:
        print("Google Drive에 파일이 없습니다.")
    else:
        print("Google Drive 파일 목록:")
        for item in items:
            print(f"{item['name']} (ID: {item['id']})")
    return items

# 4. Google Drive 파일 다운로드
def download_file(drive_service, file_id, file_name):
    request = drive_service.files().get_media(fileId=file_id)
    with io.FileIO(file_name, 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"다운로드 진행률: {int(status.progress() * 100)}%")
    print(f"파일 다운로드 완료: {file_name}")

# 5. 데이터 분석 (Pandas)
def analyze_csv(file_path):
    try:
        df = pd.read_csv(file_path, encoding='utf-8')  # 필요한 경우 'cp949'로 변경
        print(f"CSV 파일 로드 완료: {file_path}")
        print("데이터 미리보기:")
        print(df.head())

        # 간단한 분석: 열 이름과 데이터 통계
        print("\n열 이름:", df.columns.tolist())
        print("\n데이터 통계:")
        print(df.describe())

        return df
    except Exception as e:
        print(f"CSV 파일 분석 중 오류 발생: {e}")
        return None

# 6. 전체 실행
def main():
    # Step 1: 인증 파일 경로 설정
    credentials_path = set_credentials()

    # Step 2: Google Drive 인증
    drive_service = authenticate_google_drive(credentials_path)

    # Step 3: Google Drive 파일 목록 가져오기
    drive_files = list_drive_files(drive_service)

    # Step 4: 특정 파일 다운로드 및 분석 (첫 번째 파일 예시)
    if drive_files:
        target_file = drive_files[0]  # 첫 번째 파일 선택
        file_id = target_file['id']
        file_name = target_file['name']

        # 파일 다운로드
        download_file(drive_service, file_id, file_name)

        # 다운로드한 파일 분석
        if file_name.endswith('.csv'):
            analyze_csv(file_name)
        else:
            print(f"다운로드한 파일은 CSV 형식이 아닙니다: {file_name}")

# 실행
if __name__ == "__main__":
    main()

인증 파일 경로 확인 완료: credentials.json
Google Drive 파일 목록:
코인정보 (ID: 1Fla5yZ3owaIOaSfRJF8PAly_7i2itC2BILtAlZgVJtI)
유통망정산 (ID: 1gZ468AqM4wApPDMD5LZnA3WZpi2q6zqP8M4ZU_xUWcY)


HttpError: <HttpError 403 when requesting https://www.googleapis.com/drive/v3/files/1Fla5yZ3owaIOaSfRJF8PAly_7i2itC2BILtAlZgVJtI?alt=media returned "Only files with binary content can be downloaded. Use Export with Docs Editors files.". Details: "[{'message': 'Only files with binary content can be downloaded. Use Export with Docs Editors files.', 'domain': 'global', 'reason': 'fileNotDownloadable', 'location': 'alt', 'locationType': 'parameter'}]">

In [40]:
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google.oauth2.service_account import Credentials
import io
import pandas as pd
import os

# Google Drive 파일 다운로드 (Export 방식)
def download_google_sheets_as_csv(drive_service, file_id, file_name):
    # CSV 형식으로 변환하여 다운로드
    request = drive_service.files().export_media(fileId=file_id, mimeType='text/csv')
    file_path = os.path.join(os.getcwd(), file_name)
    with io.FileIO(file_path, 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"다운로드 진행률: {int(status.progress() * 100)}%")
    print(f"파일 다운로드 완료: {file_path}")
    return file_path

# 수정된 main 함수
def main():
  # 1. 인증 파일 경로 설정
   def set_credentials():
     credentials_path = "D:\\구글key\\credentials (1).json"  # 실제 credentials.json 파일 경로 입력
    if not os.path.exists(credentials_path):
        raise FileNotFoundError("credentials.json 파일을 찾을 수 없습니다. 경로를 확인하세요.")
    print("인증 파일 경로 확인 완료: credentials.json")
    return credentials_path

  # 2. Google Drive 인증
   def authenticate_google_drive(credentials_path):
    SCOPES = ['https://www.googleapis.com/auth/drive']
    creds = Credentials.from_service_account_file(credentials_path, scopes=SCOPES)
    drive_service = build('drive', 'v3', credentials=creds)
    return drive_service
    # Step 3: Google Drive 파일 목록 가져오기
    results = drive_service.files().list(
        pageSize=10, fields="nextPageToken, files(id, name, mimeType)"
    ).execute()
    files = results.get('files', [])
   
    if not files:
        print("Google Drive에 파일이 없습니다.")
        return
   
    print("Google Drive 파일 목록:")
    for file in files:
        print(f"{file['name']} (ID: {file['id']}, MIME Type: {file['mimeType']})")
   
    # Step 4: 특정 파일 다운로드 및 변환
    target_file = files[0]  # 첫 번째 파일 선택
    file_id = target_file['id']
    file_name = target_file['name']
    mime_type = target_file['mimeType']
   
    if mime_type == 'application/vnd.google-apps.spreadsheet':  # Google Sheets 파일
        print(f"{file_name} 파일은 Google Sheets 형식입니다. CSV로 변환하여 다운로드합니다.")
        file_path = download_google_sheets_as_csv(drive_service, file_id, file_name + ".csv")
       
        # 다운로드한 CSV 파일 분석
        analyze_csv(file_path)
    else:
        print(f"{file_name} 파일은 변환이 필요하지 않은 형식입니다. 직접 다운로드하세요.")

# 데이터 분석 함수
def analyze_csv(file_path):
    try:
        df = pd.read_csv(file_path)
        print("CSV 데이터 미리보기:")
        print(df.head())
    except Exception as e:
        print(f"CSV 파일 분석 중 오류 발생: {e}")

# 실행
if __name__ == "__main__":
    main()

IndentationError: unindent does not match any outer indentation level (<string>, line 27)

## 코인 정보

In [42]:
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google.oauth2.service_account import Credentials
import io
import pandas as pd
import os

# Google Drive 파일 다운로드 (Export 방식)
def download_google_sheets_as_csv(drive_service, file_id, file_name):
    # CSV 형식으로 변환하여 다운로드
    request = drive_service.files().export_media(fileId=file_id, mimeType='text/csv')
    file_path = os.path.join(os.getcwd(), file_name)
    with io.FileIO(file_path, 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"다운로드 진행률: {int(status.progress() * 100)}%")
    print(f"파일 다운로드 완료: {file_path}")
    return file_path

# 데이터 분석 함수
def analyze_csv(file_path):
    try:
        df = pd.read_csv(file_path)
        print("CSV 데이터 미리보기:")
        print(df.head())
    except Exception as e:
        print(f"CSV 파일 분석 중 오류 발생: {e}")

# 메인 함수
def main():
    # 1. 인증 파일 경로 설정
    def set_credentials():
        credentials_path = "D:\\구글key\\credentials (1).json"  # 실제 credentials.json 파일 경로 입력
        if not os.path.exists(credentials_path):
            raise FileNotFoundError("credentials.json 파일을 찾을 수 없습니다. 경로를 확인하세요.")
        print("인증 파일 경로 확인 완료: credentials.json")
        return credentials_path

    # 2. Google Drive 인증
    def authenticate_google_drive(credentials_path):
        SCOPES = ['https://www.googleapis.com/auth/drive']
        creds = Credentials.from_service_account_file(credentials_path, scopes=SCOPES)
        drive_service = build('drive', 'v3', credentials=creds)
        return drive_service

    # Step 1: 인증 파일 경로 가져오기
    credentials_path = set_credentials()

    # Step 2: Google Drive 인증
    drive_service = authenticate_google_drive(credentials_path)

    # Step 3: Google Drive 파일 목록 가져오기
    results = drive_service.files().list(
        pageSize=10, fields="nextPageToken, files(id, name, mimeType)"
    ).execute()
    files = results.get('files', [])
   
    if not files:
        print("Google Drive에 파일이 없습니다.")
        return
   
    print("Google Drive 파일 목록:")
    for file in files:
        print(f"{file['name']} (ID: {file['id']}, MIME Type: {file['mimeType']})")
   
    # Step 4: 특정 파일 다운로드 및 변환
    target_file = files[0]  # 첫 번째 파일 선택
    file_id = target_file['id']
    file_name = target_file['name']
    mime_type = target_file['mimeType']
   
    if mime_type == 'application/vnd.google-apps.spreadsheet':  # Google Sheets 파일
        print(f"{file_name} 파일은 Google Sheets 형식입니다. CSV로 변환하여 다운로드합니다.")
        file_path = download_google_sheets_as_csv(drive_service, file_id, file_name + ".csv")
       
        # 다운로드한 CSV 파일 분석
        analyze_csv(file_path)
    else:
        print(f"{file_name} 파일은 변환이 필요하지 않은 형식입니다. 직접 다운로드하세요.")

# 실행
if __name__ == "__main__":
    main()



인증 파일 경로 확인 완료: credentials.json
Google Drive 파일 목록:
코인정보 (ID: 1Fla5yZ3owaIOaSfRJF8PAly_7i2itC2BILtAlZgVJtI, MIME Type: application/vnd.google-apps.spreadsheet)
유통망정산 (ID: 1gZ468AqM4wApPDMD5LZnA3WZpi2q6zqP8M4ZU_xUWcY, MIME Type: application/vnd.google-apps.spreadsheet)
코인정보 파일은 Google Sheets 형식입니다. CSV로 변환하여 다운로드합니다.
다운로드 진행률: 100%
파일 다운로드 완료: C:\Users\User\Desktop\jupytertest\코인정보.csv
CSV 데이터 미리보기:
   2024-10-01 00:05:36       N/A  Unnamed: 2 Unnamed: 3
0  2024-10-01 00:11:45       NaN         NaN        NaN
1  2024-10-01 00:22:35   Bitcoin  $63,161.68        NaN
2  2024-10-01 00:24:24   Bitcoin  $63,161.68        NaN
3  2024-10-01 00:24:24  Ethereum   $2,599.15        NaN
4  2024-10-01 00:24:24      Mina     $0.5561        NaN


In [45]:
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google.oauth2.service_account import Credentials
import pandas as pd
import io
import os
import matplotlib.pyplot as plt

# Google Drive 파일 다운로드 (Export 방식)
def download_google_sheets_as_csv(drive_service, file_id, file_name):
    # CSV 형식으로 변환하여 다운로드
    request = drive_service.files().export_media(fileId=file_id, mimeType='text/csv')
    file_path = os.path.join(os.getcwd(), file_name)
    with io.FileIO(file_path, 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"다운로드 진행률: {int(status.progress() * 100)}%")
    print(f"파일 다운로드 완료: {file_path}")
    return file_path

# 데이터 분석 및 시각화 함수
def analyze_and_visualize(file_path):
    try:
        # CSV 데이터 로드
        df = pd.read_csv(file_path)
        print("CSV 데이터 미리보기:")
        print(df.head())
       
        # 데이터프레임 정보 확인
        print("\n데이터프레임 요약:")
        print(df.info())
       
        # 결측값 확인
        print("\n결측값 확인:")
        print(df.isnull().sum())
       
        # 데이터 요약 통계
        print("\n기초 통계:")
        print(df.describe())

        # 시각화 예제
        # 1. 특정 열의 값 분포 확인 (예: '상태' 열)
        if '상태' in df.columns:
            plt.figure(figsize=(8, 6))
            df['상태'].value_counts().plot(kind='bar', title='상태별 분포')
            plt.xlabel('상태')
            plt.ylabel('개수')
            plt.show()
       
        # 2. 시간별 데이터 추세 (예: '시간' 열이 있을 경우)
        if '시간' in df.columns:
            df['시간'] = pd.to_datetime(df['시간'])  # 시간 열을 datetime 형식으로 변환
            df.set_index('시간', inplace=True)
            df.resample('D').size().plot(title='시간별 데이터 추세')
            plt.xlabel('날짜')
            plt.ylabel('건수')
            plt.show()

    except Exception as e:
        print(f"CSV 파일 분석 중 오류 발생: {e}")

# 메인 함수
def main():
    # 1. 인증 파일 경로 설정
    def set_credentials():
        credentials_path = "D:\\구글key\\credentials (1).json"  # 실제 credentials.json 파일 경로 입력
        if not os.path.exists(credentials_path):
            raise FileNotFoundError("credentials.json 파일을 찾을 수 없습니다. 경로를 확인하세요.")
        print("인증 파일 경로 확인 완료: credentials.json")
        return credentials_path

    # 2. Google Drive 인증
    def authenticate_google_drive(credentials_path):
        SCOPES = ['https://www.googleapis.com/auth/drive']
        creds = Credentials.from_service_account_file(credentials_path, scopes=SCOPES)
        drive_service = build('drive', 'v3', credentials=creds)
        return drive_service

    # Step 1: 인증 파일 경로 가져오기
    credentials_path = set_credentials()

    # Step 2: Google Drive 인증
    drive_service = authenticate_google_drive(credentials_path)

    # Step 3: Google Drive 파일 목록 가져오기
    results = drive_service.files().list(
        pageSize=10, fields="nextPageToken, files(id, name, mimeType)"
    ).execute()
    files = results.get('files', [])
   
    if not files:
        print("Google Drive에 파일이 없습니다.")
        return
   
    print("Google Drive 파일 목록:")
    for file in files:
        print(f"{file['name']} (ID: {file['id']}, MIME Type: {file['mimeType']})")
   
    # Step 4: '일시정지관리' 파일 다운로드 및 변환
    target_file = next((f for f in files if f['name'] == '일시정지관리'), None)
    if not target_file:
        print("Google Drive에 '일시정지관리' 파일이 없습니다.")
        return

    file_id = target_file['id']
    file_name = target_file['name']
    file_path = download_google_sheets_as_csv(drive_service, file_id, file_name + ".csv")

    # Step 5: 다운로드한 CSV 파일 분석 및 시각화
    analyze_and_visualize(file_path)

# 실행
if __name__ == "__main__":
    main()


인증 파일 경로 확인 완료: credentials.json
Google Drive 파일 목록:
코인정보 (ID: 1Fla5yZ3owaIOaSfRJF8PAly_7i2itC2BILtAlZgVJtI, MIME Type: application/vnd.google-apps.spreadsheet)
유통망정산 (ID: 1gZ468AqM4wApPDMD5LZnA3WZpi2q6zqP8M4ZU_xUWcY, MIME Type: application/vnd.google-apps.spreadsheet)
Google Drive에 '일시정지관리' 파일이 없습니다.


In [47]:
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google.oauth2.service_account import Credentials
import pandas as pd
import io
import os
import matplotlib.pyplot as plt

# Google Drive 파일 다운로드 (Export 방식)
def download_google_sheets_as_csv(drive_service, file_id, file_name):
    # CSV 형식으로 변환하여 다운로드
    request = drive_service.files().export_media(fileId=file_id, mimeType='text/csv')
    file_path = os.path.join(os.getcwd(), file_name)
    with io.FileIO(file_path, 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"다운로드 진행률: {int(status.progress() * 100)}%")
    print(f"파일 다운로드 완료: {file_path}")
    return file_path

# 데이터 분석 및 시각화 함수
def analyze_and_visualize(file_path):
    try:
        # CSV 데이터 로드
        df = pd.read_csv(file_path)
        print("CSV 데이터 미리보기:")
        print(df.head())
       
        # 데이터프레임 정보 확인
        print("\n데이터프레임 요약:")
        print(df.info())
       
        # 결측값 확인
        print("\n결측값 확인:")
        print(df.isnull().sum())
       
        # 데이터 요약 통계
        print("\n기초 통계:")
        print(df.describe())

        # 시각화 예제
        # 1. 특정 열의 값 분포 확인 (예: '상태' 열)
        if '상태' in df.columns:
            plt.figure(figsize=(8, 6))
            df['상태'].value_counts().plot(kind='bar', title='상태별 분포')
            plt.xlabel('상태')
            plt.ylabel('개수')
            plt.show()
       
        # 2. 시간별 데이터 추세 (예: '시간' 열이 있을 경우)
        if '시간' in df.columns:
            df['시간'] = pd.to_datetime(df['시간'])  # 시간 열을 datetime 형식으로 변환
            df.set_index('시간', inplace=True)
            df.resample('D').size().plot(title='시간별 데이터 추세')
            plt.xlabel('날짜')
            plt.ylabel('건수')
            plt.show()

    except Exception as e:
        print(f"CSV 파일 분석 중 오류 발생: {e}")

# 메인 함수
def main():
    # 1. 인증 파일 경로 설정
    def set_credentials():
        credentials_path = "D:\\구글key\\credentials.json"  # 실제 credentials.json 파일 경로 입력
        if not os.path.exists(credentials_path):
            raise FileNotFoundError("credentials.json 파일을 찾을 수 없습니다. 경로를 확인하세요.")
        print("인증 파일 경로 확인 완료: credentials.json")
        return credentials_path

    # 2. Google Drive 인증
    def authenticate_google_drive(credentials_path):
        SCOPES = ['https://www.googleapis.com/auth/drive']
        creds = Credentials.from_service_account_file(credentials_path, scopes=SCOPES)
        drive_service = build('drive', 'v3', credentials=creds)
        return drive_service

    # Step 1: 인증 파일 경로 가져오기
    credentials_path = set_credentials()

    # Step 2: Google Drive 인증
    drive_service = authenticate_google_drive(credentials_path)

    # Step 3: Google Drive 파일 목록 가져오기
    results = drive_service.files().list(
        pageSize=100, fields="nextPageToken, files(id, name, mimeType)"
    ).execute()
    files = results.get('files', [])
   
    if not files:
        print("Google Drive에 파일이 없습니다.")
        return
   
    print("Google Drive 파일 목록:")
    for file in files:
        print(f"{file['name']} (ID: {file['id']}, MIME Type: {file['mimeType']})")
   
    # Step 4: '일시정지관리' 파일 다운로드 및 변환
    target_file = next((f for f in files if f['name'] == '일시정지관리'), None)
    if not target_file:
        print("Google Drive에 '일시정지관리' 파일이 없습니다.")
        return

    file_id = target_file['id']
    file_name = target_file['name']
    file_path = download_google_sheets_as_csv(drive_service, file_id, file_name + ".csv")

    # Step 5: 다운로드한 CSV 파일 분석 및 시각화
    analyze_and_visualize(file_path)

# 실행
if __name__ == "__main__":
    main()



인증 파일 경로 확인 완료: credentials.json
Google Drive 파일 목록:
코인정보 (ID: 1Fla5yZ3owaIOaSfRJF8PAly_7i2itC2BILtAlZgVJtI, MIME Type: application/vnd.google-apps.spreadsheet)
유통망정산 (ID: 1gZ468AqM4wApPDMD5LZnA3WZpi2q6zqP8M4ZU_xUWcY, MIME Type: application/vnd.google-apps.spreadsheet)
Google Drive에 '일시정지관리' 파일이 없습니다.


In [49]:
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google.oauth2.service_account import Credentials
import io
import pandas as pd
import os
import matplotlib.pyplot as plt

# Google Drive 파일 다운로드 (Export 방식)
def download_google_sheets_as_csv(drive_service, file_id, file_name):
    # CSV 형식으로 변환하여 다운로드
    request = drive_service.files().export_media(fileId=file_id, mimeType='text/csv')
    file_path = os.path.join(os.getcwd(), file_name)
    with io.FileIO(file_path, 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"다운로드 진행률: {int(status.progress() * 100)}%")
    print(f"파일 다운로드 완료: {file_path}")
    return file_path

# 데이터 분석 및 시각화 함수
def analyze_and_visualize(file_path):
    try:
        # CSV 데이터 로드
        df = pd.read_csv(file_path)
        print("CSV 데이터 미리보기:")
        print(df.head())
       
        # 데이터프레임 정보 확인
        print("\n데이터프레임 요약:")
        print(df.info())
       
        # 결측값 확인
        print("\n결측값 확인:")
        print(df.isnull().sum())
       
        # 데이터 요약 통계
        print("\n기초 통계:")
        print(df.describe())

        # 시각화 예제
        # 1. '관리지사'별 계약 수 분포
        if '관리지사' in df.columns:
            plt.figure(figsize=(10, 6))
            df['관리지사'].value_counts().plot(kind='bar', title='관리지사별 계약 수')
            plt.xlabel('관리지사')
            plt.ylabel('계약 수')
            plt.show()

        # 2. 서비스(소)별 분포
        if '서비스(소)' in df.columns:
            plt.figure(figsize=(10, 6))
            df['서비스(소)'].value_counts().plot(kind='bar', title='서비스(소)별 분포')
            plt.xlabel('서비스(소)')
            plt.ylabel('개수')
            plt.show()

    except Exception as e:
        print(f"CSV 파일 분석 중 오류 발생: {e}")

# 메인 함수
def main():
    # 1. 인증 파일 경로 설정
    def set_credentials():
        credentials_path = "D:\\구글key\\credentials.json"  # 실제 credentials.json 파일 경로 입력
        if not os.path.exists(credentials_path):
            raise FileNotFoundError("credentials.json 파일을 찾을 수 없습니다. 경로를 확인하세요.")
        print("인증 파일 경로 확인 완료: credentials.json")
        return credentials_path

    # 2. Google Drive 인증
    def authenticate_google_drive(credentials_path):
        SCOPES = ['https://www.googleapis.com/auth/drive']
        creds = Credentials.from_service_account_file(credentials_path, scopes=SCOPES)
        drive_service = build('drive', 'v3', credentials=creds)
        return drive_service

    # Step 1: 인증 파일 경로 가져오기
    credentials_path = set_credentials()

    # Step 2: Google Drive 인증
    drive_service = authenticate_google_drive(credentials_path)

    # Step 3: Google Drive 파일 목록 가져오기
    results = drive_service.files().list(
        pageSize=100, fields="nextPageToken, files(id, name, mimeType)"
    ).execute()
    files = results.get('files', [])
   
    if not files:
        print("Google Drive에 파일이 없습니다.")
        return
   
    print("Google Drive 파일 목록:")
    for file in files:
        print(f"{file['name']} (ID: {file['id']}, MIME Type: {file['mimeType']})")
   
    # Step 4: '일시정지관리' 파일 다운로드 및 변환
    target_file = next((f for f in files if f['name'] == '일시정지관리'), None)
    if not target_file:
        print("Google Drive에 '일시정지관리' 파일이 없습니다.")
        return

    file_id = target_file['id']
    file_name = target_file['name']
    file_path = download_google_sheets_as_csv(drive_service, file_id, file_name + ".csv")

    # Step 5: 다운로드한 CSV 파일 분석 및 시각화
    analyze_and_visualize(file_path)

# 실행
if __name__ == "__main__":
    main()

인증 파일 경로 확인 완료: credentials.json
Google Drive 파일 목록:
코인정보 (ID: 1Fla5yZ3owaIOaSfRJF8PAly_7i2itC2BILtAlZgVJtI, MIME Type: application/vnd.google-apps.spreadsheet)
유통망정산 (ID: 1gZ468AqM4wApPDMD5LZnA3WZpi2q6zqP8M4ZU_xUWcY, MIME Type: application/vnd.google-apps.spreadsheet)
Google Drive에 '일시정지관리' 파일이 없습니다.


In [1]:
import pandas as pd

# 파일 경로 설정, 확장자가 .cs
file_path = 'D:/시설/정지/일일정지/고객리스트 정지시설 전체/C150_G0000_00024.csv'

# CSV 파일 읽기, 헤더 및 데이터 타입 경고 처리
df = pd.read_csv(file_path, encoding='cp949', low_memory=False)

# 헤더만 출력, 보기 좋게 줄바꿈하여 표시
for column in df.columns:
    print(column)

관리본부코드
관리본부명
관리지사코드
관리지사명
고객번호
고객명
계약번호
유지보수기간
계약자명
서비스번호
서비스(대)
서비스(중)
서비스(소)
영상전환일
상호
고객구분
사업용구분
주민등록번호
사업자번호
계약상태(대)
계약상태(중)
서비스상태(대)
서비스상태(중)
접속전화번호
설치우편번호
설치주소
견적월정료
KTT월정료
KT월정료
할인율(%)
계약개월수
월정료 면제사유
면제시작일
면제종료일
계약보증금
보증금 면제사유
판매상품비
견적설치공사비
설치공사할부개월
계약공사비(ktt)
계약공사비(kt)
면제공사비
실징수액
계약시작일
계약종료일
영업구분
추천구분
추천경로
영업본부코드
영업본부명
영업지사코드
영업지사명
영업자소속
영업자사번
영업자명
영업자연락처
모집유형
가망고객번호
추천본부코드
추천본부명
추천지사코드
추천지사명
유통망대분류
유통망중분류
유통망소분류
추천자사번
추천자명
추천자유형
정보제공자부서
정보제공자사번
정보제공자명
정보제공자연락처
청구본부코드
청구본부명
청구지사코드
청구지사명
청구자명
합산여부
수납구분
세금계산서발행여부
계산서발행여부
선후납구분
청구번호
청구전화번호
청구우편번호
청구주소
청약일자
청약취소일자
공사희망일
공사예정일
공사완료일
준공완료일
관제개통일
서비스개시일
서비스재개시일
공사의뢰유형
회선방식(대)
회선방식(중)
MUX / SYSTEM_ID
전용회선번호
MIN번호
정지시작일자
해지일자
종목
업태
업종(대)
업종(중)
보험등급
비고
결합구분
결합상품명
결합약정개월수
결합시작일
금융기관코드
면책구분
타사전환
이전고객번호
E-MAIL
담당자
휴대폰
전입신규여부
고객추가정보
계약최초서비스게시일
청약취소사유
계약본부
계약지사
매출본사
출동구역정보
영업구역정보
구역담당지점
구역담당영업사원
선장품
브랜드
보조회선
고객계약유형
A/S운용기간
판매할부개월
월정료일시납
장기요금인상여부
장기요금인상액
장기Upselling여부
장기Upselling매출액
기술구역정보
계약서작성주체
카드사명
GiGAeyes계약Id
월정료
kt oct 마스터 아이디
정지희망종료일
보조

In [3]:
import pandas as pd

# 파일 경로 설정 (확장자 .cs)
file_path = 'D:/시설/정지/일일정지/고객리스트 정지시설 전체/C150_G0000_00024.csv'

# CSV 파일 읽기 (헤더 및 데이터 타입 경고 처리)
df = pd.read_csv(file_path, encoding='cp949', low_memory=False)

# 추출할 헤더(열) 지정
columns_to_extract = ['관리본부명', '관리지사명', '계약번호', '서비스(중)', '서비스(소)', '상호',
    '고객구분', '사업용구분', '계약상태(중)', '서비스상태(중)',
    '설치주소', '월정료', '계약시작일', '영업자명', '영업채널', '청구번호', '서비스개시일',
    '서비스재개시일', '정지시작일자', '종목', 'E-MAIL', '담당자', '휴대폰', '계약최초서비스게시일', '매출본사',
    '매출인정구분', '시설구분', '요금구분', '고객계약유형', '실적본부', '실적지사',
    '실적최초등록일', '실적구분', '영업구역정보', '매출구분', '영업본부2', '영업지사2', '실적채널', '실적부서',
    'KTT월정료(조정)', 'KT월정료(조정)',
    '고알프', '제외사유']

# 지정된 헤더만 추출
extracted_data = df[columns_to_extract]

# "관리본부명" 값 변경: "강원본부"를 "강북강원본부"로 변경
filtered_data['관리본부명'] = filtered_data['관리본부명'].replace('강원본부', '강북/강원본부')
filtered_data['관리본부명'] = filtered_data['관리본부명'].replace('서부본부', '강남/서부본부')


# 추출한 데이터 저장 (엑셀 파일로)
output_path = 'D:/시설/정지/일일정지/고객리스트 정지시설 전체/2025년0108_전사 정지조건리스트_db.xlsx'
extracted_data.to_excel(output_path, index=False, engine='openpyxl')  # openpyxl 엔진 사용

# 추출한 데이터 표시
print("지정된 헤더 데이터가 저장되었습니다:", output_path)
display(extracted_data)

KeyError: "['영업채널', '매출인정구분', '시설구분', '요금구분', '실적본부', '실적지사', '실적최초등록일', '실적구분', '매출구분', '영업본부2', '영업지사2', '실적채널', '실적부서', 'KTT월정료(조정)', 'KT월정료(조정)', '고알프', '제외사유'] not in index"

In [5]:
import pandas as pd
import os

# 파일 경로 설정 (윈도우 드라이브 경로)
file_path = r"D:\시설\정지\일일정지\20250131\G000_02995_00008.csv"
output_excel_path = r"D:\시설\정지\일일정지\20250131\filtered_headers.xlsx"

try:
    # CSV 파일 불러오기 (인코딩 자동 감지)
    df = pd.read_csv(file_path, encoding="utf-8", low_memory=False)
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding="cp949", low_memory=False)
except Exception as e:
    print(f"파일 로드 실패: {e}")
    exit()

# 헤더 리스트 가져오기
headers = df.columns.tolist()
print("계약번호", headers)

# 특정 조건에 맞는 헤더만 선택 (예: '정지' 포함하는 컬럼)
condition_headers = [col for col in headers if "정지" in col]

# 결과 저장할 DataFrame 생성
filtered_df = pd.DataFrame({"Filtered Headers": condition_headers})

# Excel 파일로 저장
filtered_df.to_excel(output_excel_path, index=False)

print(f"필터링된 헤더를 {output_excel_path}에 저장 완료!")

계약번호 ['관리본부코드', '관리본부명', '관리지사코드', '관리지사명', '고객번호', '고객명', '계약번호', '유지보수기간', '계약자명', '서비스번호', '서비스(대)', '서비스(중)', '서비스(소)', '영상전환일', '상호', '고객구분', '사업용구분', '주민등록번호', '사업자번호', '계약상태(대)', '계약상태(중)', '서비스상태(대)', '서비스상태(중)', '접속전화번호', '설치우편번호', '설치주소', '견적월정료', 'KTT월정료', 'KT월정료', '할인율(%)', '계약개월수', '월정료 면제사유', '면제시작일', '면제종료일', '계약보증금', '보증금 면제사유', '판매상품비', '견적설치공사비', '설치공사할부개월', '계약공사비(ktt)', '계약공사비(kt)', '면제공사비', '실징수액', '계약시작일', '계약종료일', '영업구분', '추천구분', '추천경로', '영업본부코드', '영업본부명', '영업지사코드', '영업지사명', '영업자소속', '영업자사번', '영업자명', '영업자연락처', '모집유형', '가망고객번호', '추천본부코드', '추천본부명', '추천지사코드', '추천지사명', '유통망대분류', '유통망중분류', '유통망소분류', '추천자사번', '추천자명', '추천자유형', '정보제공자부서', '정보제공자사번', '정보제공자명', '정보제공자연락처', '청구본부코드', '청구본부명', '청구지사코드', '청구지사명', '청구자명', '합산여부', '수납구분', '세금계산서발행여부', '계산서발행여부', '선후납구분', '청구번호', '청구전화번호', '청구우편번호', '청구주소', '청약일자', '청약취소일자', '공사희망일', '공사예정일', '공사완료일', '준공완료일', '관제개통일', '서비스개시일', '서비스재개시일', '공사의뢰유형', '회선방식(대)', '회선방식(중)', 'MUX / SYSTEM_ID', '전용회선번호', 'MIN번호', '정지시작일자', '해지일자', '종

In [9]:
import pandas as pd

# 파일 경로 설정, 확장자가 .cs
file_path = 'D:/시설/정지/일일정지/20250131/G000_02995_00008.csv'

# CSV 파일 읽기, 헤더 및 데이터 타입 경고 처리
df = pd.read_csv(file_path, encoding='cp949', low_memory=False)

# 헤더만 출력, 보기 좋게 줄바꿈하여 표시
for column in df.columns:
    print(column)

관리본부코드
관리본부명
관리지사코드
관리지사명
고객번호
고객명
계약번호
유지보수기간
계약자명
서비스번호
서비스(대)
서비스(중)
서비스(소)
영상전환일
상호
고객구분
사업용구분
주민등록번호
사업자번호
계약상태(대)
계약상태(중)
서비스상태(대)
서비스상태(중)
접속전화번호
설치우편번호
설치주소
견적월정료
KTT월정료
KT월정료
할인율(%)
계약개월수
월정료 면제사유
면제시작일
면제종료일
계약보증금
보증금 면제사유
판매상품비
견적설치공사비
설치공사할부개월
계약공사비(ktt)
계약공사비(kt)
면제공사비
실징수액
계약시작일
계약종료일
영업구분
추천구분
추천경로
영업본부코드
영업본부명
영업지사코드
영업지사명
영업자소속
영업자사번
영업자명
영업자연락처
모집유형
가망고객번호
추천본부코드
추천본부명
추천지사코드
추천지사명
유통망대분류
유통망중분류
유통망소분류
추천자사번
추천자명
추천자유형
정보제공자부서
정보제공자사번
정보제공자명
정보제공자연락처
청구본부코드
청구본부명
청구지사코드
청구지사명
청구자명
합산여부
수납구분
세금계산서발행여부
계산서발행여부
선후납구분
청구번호
청구전화번호
청구우편번호
청구주소
청약일자
청약취소일자
공사희망일
공사예정일
공사완료일
준공완료일
관제개통일
서비스개시일
서비스재개시일
공사의뢰유형
회선방식(대)
회선방식(중)
MUX / SYSTEM_ID
전용회선번호
MIN번호
정지시작일자
해지일자
종목
업태
업종(대)
업종(중)
보험등급
비고
결합구분
결합상품명
결합약정개월수
결합시작일
금융기관코드
면책구분
타사전환
이전고객번호
E-MAIL
담당자
휴대폰
전입신규여부
고객추가정보
계약최초서비스게시일
청약취소사유
계약본부
계약지사
매출본사
출동구역정보
영업구역정보
구역담당지점
구역담당영업사원
선장품
브랜드
보조회선
고객계약유형
A/S운용기간
판매할부개월
월정료일시납
장기요금인상여부
장기요금인상액
장기Upselling여부
장기Upselling매출액
기술구역정보
계약서작성주체
카드사명
GiGAeyes계약Id
월정료
kt oct 마스터 아이디
정지희망종료일
보조

In [3]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Alignment

# 파일 경로 설정
file_path = r'D:\시설\정지\일일정지\20250209\C150_G0000_00019.csv'
output_excel_path = r'D:\시설\정지\일일정지\20250209\2025년0209_전사 정지조건리스트_db.xlsx'
output_cs_path = r'D:\시설\정지\일일정지\20250209\2025년0209_전사 정지조건리스트_db.cs'  # CSV 대신 CS 확장자로 저장

try:
    # CSV 파일 읽기 (인코딩 처리 및 경고 방지)
    df = pd.read_csv(file_path, encoding='cp949', low_memory=False)
   
    # 추출할 헤더(열) 지정
    columns_to_extract = ['관리본부명', '관리지사명', '계약번호',
                          '서비스(대)', '서비스(중)', '서비스(소)', '상호', '고객구분', '사업용구분', '계약상태(대)', '설치주소', 'KTT월정료',
                          '계약시작일', '계약종료일',
                          '영업자명', '정지시작일자', '정지희망종료일', '담당자', '휴대폰', '계약최초서비스게시일',
                          '영업구역정보','kt oct 마스터 아이디', '외부고객KEY', '무인매장구분',
                          '무인매장업종구분']
   
    # 데이터에서 지정된 헤더만 추출
    extracted_data = df[columns_to_extract].copy()  # 복사하여 수정 가능하도록 함

    # "관리본부명" 값 변경
    extracted_data['관리본부명'] = extracted_data['관리본부명'].replace({
        '강원본부': '강북/강원본부',
        '서부본부': '강남/서부본부'
    })

    # 🔹 서비스(소)가 공백(NaN)인 경우 서비스(중) 값으로 대체
    extracted_data['서비스(소)'] = extracted_data['서비스(소)'].fillna(extracted_data['서비스(중)'])

    # 날짜 변환 함수 정의
    def convert_date(date_str):
        """ YYYYMMDD 또는 YYYYMMDDHHMMSS 형식을 YYYY-MM-DD로 변환 """
        try:
            date_str = str(date_str).strip()  # 문자열 변환 후 공백 제거
            if len(date_str) >= 8:
                return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}"  # YYYY-MM-DD 변환
            else:
                return None  # 변환 불가능한 값은 None 처리
        except:
            return None

    # 날짜 변환 적용할 필드 목록
    date_columns = ['계약시작일', '계약종료일', '정지시작일자', '정지희망종료일', '계약최초서비스게시일']

    # 각 날짜 컬럼 변환 적용
    for col in date_columns:
        extracted_data[col] = extracted_data[col].astype(str).apply(convert_date)

    # 천단위 콤마 변환 함수 정의 (NaN 또는 0은 공백)
    def format_currency(value):
        """ 숫자를 천단위 콤마 적용한 문자열로 변환 (예: 60000 -> 60,000, NaN & 0 -> 공백) """
        try:
            if pd.isna(value) or value in ["0", "0.0"]:  # NaN 또는 0이면 공백 처리
                return ""
            return int(float(value))  # 엑셀에서 회계 형식 적용 가능하도록 숫자로 변환
        except:
            return value  # 변환 실패 시 원본 값 유지

    # 천단위 콤마 적용할 필드 목록
    currency_columns = ['KTT월정료']

    # 각 숫자 컬럼 변환 적용 (NaN 및 0을 공백으로 변환)
    for col in currency_columns:
        extracted_data[col] = extracted_data[col].astype(str).apply(format_currency)

    # 엑셀 파일로 저장 (일반 저장)
    extracted_data.to_excel(output_excel_path, index=False, engine='openpyxl')

    # 🔹 엑셀에서 회계 형식 적용 (₩ 제외, 천단위 콤마 유지)
    wb = load_workbook(output_excel_path)
    ws = wb.active

    for col in ws.iter_cols():
        if col[0].value in currency_columns:  # 월정료 관련 컬럼만 적용
            for cell in col[1:]:  # 헤더 제외, 데이터 행부터 적용
                if isinstance(cell.value, (int, float)):  # 숫자만 서식 적용
                    cell.number_format = '#,##0'  # 원화(₩) 없이 천단위 콤마 적용
                    cell.alignment = Alignment(horizontal='right')  # 오른쪽 정렬

    # 수정된 엑셀 저장
    wb.save(output_excel_path)

    # 🔹 CS 확장자로 CSV 저장 (쉼표로 구분)
    extracted_data.to_csv(output_cs_path, index=False, encoding='cp949')

    print("✅ 엑셀 및 CS 파일이 저장되었습니다:")
    print(f" - 엑셀 파일: {output_excel_path}")
    print(f" - CS 파일: {output_cs_path}")

except Exception as e:
    print("오류 발생:", e)

✅ 엑셀 및 CS 파일이 저장되었습니다:
 - 엑셀 파일: D:\시설\정지\일일정지\20250209\2025년0209_전사 정지조건리스트_db.xlsx
 - CS 파일: D:\시설\정지\일일정지\20250209\2025년0209_전사 정지조건리스트_db.cs


## 일수, 일수구간, 금액구간

In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Alignment
from datetime import datetime, timedelta

# 파일 경로 설정
file_path = r'D:\시설\정지\일일정지\20250209\C150_G0000_00019.csv'
output_excel_path = r'D:\시설\정지\일일정지\20250209\2025년0209_전사 정지조건리스트_db_일수,금액구간적용.xlsx'
output_cs_path = r'D:\시설\정지\일일정지\20250209\2025년0209_전사 정지조건리스트_db_일수,금액구간적용.cs'  # CSV 대신 CS 확장자로 저장

try:
    # CSV 파일 읽기 (인코딩 처리 및 경고 방지)
    df = pd.read_csv(file_path, encoding='cp949', low_memory=False)

    # ✅ **오류 수정: 모든 문자열에 따옴표(`'`) 추가**
    columns_to_extract = [
        '관리본부명', '관리지사명', '계약번호', '서비스(대)', '서비스(중)', '서비스(소)', '상호',
        '고객구분', '사업용구분', '계약상태(대)', '설치주소', 'KTT월정료', '계약시작일', '계약종료일',
        '영업자명', '정지시작일자', '정지희망종료일', '담당자', '휴대폰', '계약최초서비스게시일',
        '영업구역정보', 'kt oct 마스터 아이디', '외부고객KEY', '무인매장구분', '무인매장업종구분'
    ]

    # 데이터에서 지정된 헤더만 추출
    extracted_data = df[columns_to_extract].copy()

    # "관리본부명" 값 변경
    extracted_data['관리본부명'] = extracted_data['관리본부명'].replace({
        '강원본부': '강북/강원본부',
        '서부본부': '강남/서부본부'
    })

    # 🔹 서비스(소)가 공백(NaN)인 경우 서비스(중) 값으로 대체
    extracted_data['서비스(소)'] = extracted_data['서비스(소)'].fillna(extracted_data['서비스(중)'])

    # 날짜 변환 함수 정의
    def convert_date(date_str):
        """YYYYMMDD 또는 YYYYMMDDHHMMSS 형식을 YYYY-MM-DD로 변환"""
        try:
            date_str = str(date_str).strip()
            if len(date_str) >= 8:
                return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}"
            else:
                return None
        except:
            return None

    # 날짜 변환 적용할 필드 목록
    date_columns = ['계약시작일', '계약종료일', '정지시작일자', '정지희망종료일', '계약최초서비스게시일']

    # 각 날짜 컬럼 변환 적용
    for col in date_columns:
        extracted_data[col] = extracted_data[col].astype(str).apply(convert_date)

    # 현재 실행하는 달의 말일 계산 (예: 2월이면 2월 28일 또는 29일)
    today = datetime.today()
    last_day_of_month = (today.replace(day=1) + timedelta(days=31)).replace(day=1) - timedelta(days=1)

    # 🔹 정지일수 계산 함수
    def calculate_freeze_days(start_date, end_date, reference_end_date):
        """정지시작일자부터 종료일까지의 일수를 계산, 종료일이 없으면 기준 종료일 사용"""
        try:
            if pd.isna(start_date):
                return None
            start_date = datetime.strptime(start_date, "%Y-%m-%d")

            # 종료일이 없으면 기준 종료일(현재 달의 말일) 사용
            end_date = datetime.strptime(end_date, "%Y-%m-%d") if pd.notna(end_date) else reference_end_date

            freeze_days = (end_date - start_date).days + 1
            return freeze_days if freeze_days > 0 else 0
        except:
            return None

    # 종료기준 정지일수 (정지희망종료일까지 계산)
    extracted_data['종료기준 정지일수'] = extracted_data.apply(
        lambda row: calculate_freeze_days(row['정지시작일자'], row['정지희망종료일'], last_day_of_month), axis=1
    )

    # 당월기준 정지일수 (당월 말일까지 계산)
    extracted_data['당월기준 정지일수'] = extracted_data.apply(
        lambda row: calculate_freeze_days(row['정지시작일자'], last_day_of_month.strftime("%Y-%m-%d"), last_day_of_month), axis=1
    )

    # 🔹 일시정지 구간 추가 (종료기준 & 당월기준)
    def categorize_freeze_days(days):
        """일시정지 일수를 기준으로 구간을 설정"""
        if pd.isna(days):
            return None
        elif days <= 89:
            return '89일 이하'
        elif 90 <= days <= 119:
            return '90~119일'
        elif 120 <= days <= 149:
            return '120~149일'
        else:
            return '150일 이상'

    # 종료기준 정지구간
    extracted_data['종료기준 정지구간'] = extracted_data['종료기준 정지일수'].apply(categorize_freeze_days)

    # 당월기준 정지구간
    extracted_data['당월기준 정지구간'] = extracted_data['당월기준 정지일수'].apply(categorize_freeze_days)

    # 🔹 KTT월정료 금액 구간 추가
    def categorize_fee(amount):
        """KTT월정료를 기준으로 금액 구간을 설정"""
        try:
            amount = float(amount)
            if amount <= 50000:
                return '5만원 이하'
            elif 50000 < amount <= 70000:
                return '5만원 초과 ~ 7만원 이하'
            elif 70000 < amount <= 100000:
                return '7만원 초과 ~ 10만원 이하'
            elif 100000 < amount <= 200000:
                return '10만원 초과 ~ 20만원 이하'
            else:
                return '20만원 초과'
        except:
            return None

    extracted_data['월정료 구간'] = extracted_data['KTT월정료'].apply(categorize_fee)

    # 엑셀 파일로 저장
    extracted_data.to_excel(output_excel_path, index=False, engine='openpyxl')

    # 🔹 CS 확장자로 CSV 저장
    extracted_data.to_csv(output_cs_path, index=False, encoding='cp949')

    print("✅ 엑셀 및 CS 파일이 저장되었습니다:")
    print(f" - 엑셀 파일: {output_excel_path}")
    print(f" - CS 파일: {output_cs_path}")

except Exception as e:
    print("오류 발생:", e)

## 정지파일, 유지파일 조건 적용

In [29]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from datetime import datetime, timedelta

# 📌 파일 경로 설정
file_path = r'D:\시설\정지\일일정지\20250209\C150_G0000_00019.csv'  # 기존 데이터
additional_file_path = r"D:\시설\정지\일일정지\고객리스트 정지시설 전체\'24년12월 조건요약.cs"  # 추가 데이터
output_excel_path = r'D:\시설\정지\일일정지\20250209\2025년0209_전사 정지조건리스트_완성.xlsx'
output_cs_path = r'D:\시설\정지\일일정지\20250209\2025년0209_전사 정지조건리스트_완성.cs'  # CSV 대신 CS 확장자로 저장

try:
    # 📌 기존 데이터 불러오기
    df = pd.read_csv(file_path, encoding='cp949', low_memory=False)

    # 📌 필요한 컬럼만 추출
    columns_to_extract = [
        '관리본부명', '관리지사명', '계약번호', '서비스(대)', '서비스(중)', '서비스(소)', '상호',
        '고객구분', '사업용구분', '계약상태(대)', '설치주소', 'KTT월정료', '계약시작일', '계약종료일',
        '영업자명', '정지시작일자', '정지희망종료일', '담당자', '휴대폰', '계약최초서비스게시일',
        '영업구역정보', 'kt oct 마스터 아이디', '외부고객KEY', '무인매장구분', '무인매장업종구분'
    ]
    df = df[columns_to_extract]

    # 📌 추가 데이터 불러오기
    additional_df = pd.read_csv(additional_file_path, encoding='cp949', low_memory=False)

    # 📌 필요한 컬럼 선택 (계약번호 기준으로 병합)
    additional_columns = ['계약번호', '시설구분', '요금구분', '제외사유', '매출구분', '실적채널', '고알프']
    additional_df = additional_df[additional_columns]

    # 📌 계약번호 기준으로 추가 데이터 병합 (LEFT JOIN)
    df = df.merge(additional_df, on='계약번호', how='left')

    # 📌 새 열을 기존 데이터의 맨 왼쪽으로 이동
    new_columns = additional_columns + columns_to_extract
    df = df[new_columns]

    # 📌 "관리본부명" 값 변경
    df['관리본부명'] = df['관리본부명'].replace({
        '강원본부': '강북/강원본부',
        '서부본부': '강남/서부본부'
    })

    # 📌 서비스(소)가 공백(NaN)인 경우 서비스(중) 값으로 대체
    df['서비스(소)'] = df['서비스(소)'].fillna(df['서비스(중)'])

    # 📌 날짜 변환 함수
    def convert_date(date_str):
        """YYYYMMDD 또는 YYYYMMDDHHMMSS 형식을 YYYY-MM-DD로 변환"""
        try:
            date_str = str(date_str).strip()
            if len(date_str) >= 8:
                return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}"
            else:
                return None
        except:
            return None

    # 📌 날짜 변환 적용할 필드 목록
    date_columns = ['계약시작일', '계약종료일', '정지시작일자', '정지희망종료일', '계약최초서비스게시일']
    for col in date_columns:
        df[col] = df[col].astype(str).apply(convert_date)

 # 📌 현재 실행하는 달의 말일 계산
    today = datetime.today()
    last_day_of_month = (today.replace(day=1) + timedelta(days=31)).replace(day=1) - timedelta(days=1)

    # 📌 정지일수 계산 함수
    def calculate_freeze_days(start_date, end_date, reference_end_date):
        try:
            if pd.isna(start_date):
                return None
            start_date = datetime.strptime(start_date, "%Y-%m-%d")
            end_date = datetime.strptime(end_date, "%Y-%m-%d") if pd.notna(end_date) else reference_end_date
            freeze_days = (end_date - start_date).days + 1
            return freeze_days if freeze_days > 0 else 0
        except:
            return None

    # 📌 종료기준 정지일수 (정지희망종료일까지 계산)
    df['종료기준 정지일수'] = df.apply(
        lambda row: calculate_freeze_days(row['정지시작일자'], row['정지희망종료일'], last_day_of_month), axis=1
    )

    # 📌 당월기준 정지일수 (당월 말일까지 계산)
    df['당월기준 정지일수'] = df.apply(
        lambda row: calculate_freeze_days(row['정지시작일자'], last_day_of_month.strftime("%Y-%m-%d"), last_day_of_month), axis=1
    )

    # 📌 정지구간 분류 함수
    def categorize_freeze_days(days):
        if pd.isna(days):
            return None
        elif days <= 89:
            return '89일 이하'
        elif 90 <= days <= 119:
            return '90~119일'
        elif 120 <= days <= 149:
            return '120~149일'
        else:
            return '150일 이상'

    # 📌 종료기준 및 당월기준 정지구간 추가
    df['종료기준 정지구간'] = df['종료기준 정지일수'].apply(categorize_freeze_days)
    df['당월기준 정지구간'] = df['당월기준 정지일수'].apply(categorize_freeze_days)

    # 📌 KTT월정료 금액 구간 추가
    def categorize_fee(amount):
        try:
            amount = float(amount)
            if amount <= 50000:
                return '5만원 이하'
            elif 50000 < amount <= 70000:
                return '5만원 초과 ~ 7만원 이하'
            elif 70000 < amount <= 100000:
                return '7만원 초과 ~ 10만원 이하'
            elif 100000 < amount <= 200000:
                return '10만원 초과 ~ 20만원 이하'
            else:
                return '20만원 초과'
        except:
            return None

    df['월정료 구간'] = df['KTT월정료'].apply(categorize_fee)
   
    # 📌 엑셀 파일로 저장
    df.to_excel(output_excel_path, index=False, engine='openpyxl')

    # 📌 엑셀 파일 열어서 새 열을 노란색 음영(50%) 적용
    wb = load_workbook(output_excel_path)
    ws = wb.active

    yellow_fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")

    # 새로 추가된 열에 노란색 음영 적용
    for col_num in range(1, len(additional_columns) + 1):  # 첫 번째 열부터 적용
        for row_num in range(1, ws.max_row + 1):
            ws.cell(row=row_num, column=col_num).fill = yellow_fill

    
    # 저장
    wb.save(output_excel_path)

    # 📌 CS 확장자로 CSV 저장
    df.to_csv(output_cs_path, index=False, encoding='cp949')

    print("✅ 엑셀 및 CS 파일이 저장되었습니다 (새 열은 맨 왼쪽, 노란색 음영 적용):")
    print(f" - 엑셀 파일: {output_excel_path}")
    print(f" - CS 파일: {output_cs_path}")

except Exception as e:
    print("오류 발생:", e)


✅ 엑셀 및 CS 파일이 저장되었습니다 (새 열은 맨 왼쪽, 노란색 음영 적용):
 - 엑셀 파일: D:\시설\정지\일일정지\20250209\2025년0209_전사 정지조건리스트_완성.xlsx
 - CS 파일: D:\시설\정지\일일정지\20250209\2025년0209_전사 정지조건리스트_완성.cs


## ktt월정료 공백, 제외사유 있음 제외

In [35]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from datetime import datetime, timedelta

# 📌 파일 경로 설정
file_path = r'D:\시설\정지\일일정지\20250212\C150_G0000_00072.csv'  # 기존 데이터
additional_file_path = r"D:\시설\정지\일일정지\고객리스트 정지시설 전체\'24년12월 조건요약.cs"  # 추가 데이터
output_excel_path = r'D:\시설\정지\일일정지\20250212\2025년0212_전사 정지조건리스트_완성.xlsx'
output_cs_path = r'D:\시설\정지\일일정지\20250212\2025년0212_전사 정지조건리스트_완성.cs'  # CSV 대신 CS 확장자로 저장

try:
    # 📌 기존 데이터 불러오기 (행 개수 유지)
    df = pd.read_csv(file_path, encoding='cp949', low_memory=False)
    original_row_count = df.shape[0]  # 원래 행 개수 저장 (예: 5884)

    # 📌 필요한 컬럼만 추출
    columns_to_extract = [
        '관리본부명', '관리지사명', '계약번호', '서비스(중)', '서비스(소)', '상호',
        '고객구분', '사업용구분', '계약상태(대)', '설치주소', 'KTT월정료', '계약시작일', '계약종료일',
        '영업자명', '정지시작일자', '정지희망종료일', '계약최초서비스게시일',
        '영업구역정보', 'kt oct 마스터 아이디', '외부고객KEY'
    ]
    df = df[columns_to_extract]

    # 📌 추가 데이터 불러오기
    additional_df = pd.read_csv(additional_file_path, encoding='cp949', low_memory=False)

    # 📌 중복 제거 (계약번호가 중복되면 첫 번째 값만 사용)
    additional_df = additional_df.drop_duplicates(subset=['계약번호'])

    # 📌 필요한 컬럼 선택 (계약번호 기준으로 매핑)
    additional_columns = ['시설구분', '요금구분', '제외사유', '매출구분', '실적채널', '고알프']
    additional_df = additional_df[['계약번호'] + additional_columns]

    # 📌 추가 데이터 매핑 (병합이 아닌 기존 데이터에 조건 추가)
    for col in additional_columns:
        df[col] = df['계약번호'].map(additional_df.set_index('계약번호')[col])

    # 📌 병합 후 행 개수 검증 (5884 유지)
    assert df.shape[0] == original_row_count, "매핑 후 행 개수가 변경되었습니다!"

    # 📌 날짜 변환 함수
    def convert_date(date_str):
        """YYYYMMDD 또는 YYYYMMDDHHMMSS 형식을 YYYY-MM-DD로 변환"""
        try:
            date_str = str(date_str).strip()
            if len(date_str) >= 8:
                return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}"
            else:
                return None
        except:
            return None

    # 📌 날짜 변환 적용할 필드 목록
    date_columns = ['계약시작일', '계약종료일', '정지시작일자', '정지희망종료일', '계약최초서비스게시일']
    for col in date_columns:
        df[col] = df[col].astype(str).apply(convert_date)

    # 현재 날짜
    today = datetime.today()

    # 📌 종료희망일 초과 여부 추가
    def check_exceeding_end_date(start_date, end_date):
        """현재 날짜를 기준으로 정지희망종료일 초과 여부 판단"""
        try:
            if pd.isna(start_date) or pd.isna(end_date):
                return None
            start_date = datetime.strptime(start_date, "%Y-%m-%d")
            end_date = datetime.strptime(end_date, "%Y-%m-%d")

            if end_date < today:
                return '초과'
            elif (today - start_date).days > 500:
                return '확인대상'
            else:
                return '미초과'
        except:
            return None

    df['종료희망일 초과 여부'] = df.apply(
        lambda row: check_exceeding_end_date(row['정지시작일자'], row['정지희망종료일']), axis=1
    )

    # 📌 새 열을 기존 데이터의 맨 왼쪽으로 이동
    new_columns = additional_columns + ['종료희망일 초과 여부'] + columns_to_extract
    df = df[new_columns]

    # 📌 엑셀 파일로 저장
    df.to_excel(output_excel_path, index=False, engine='openpyxl')

    # 📌 엑셀 파일 열어서 새 열을 노란색 음영(50%) 적용
    wb = load_workbook(output_excel_path)
    ws = wb.active

    yellow_fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")

    # 새로 추가된 열에 노란색 음영 적용
    for col_num in range(1, len(additional_columns) + 2):  # 추가된 열 개수만큼 적용
        for row_num in range(1, ws.max_row + 1):
            ws.cell(row=row_num, column=col_num).fill = yellow_fill

    # 저장
    wb.save(output_excel_path)

    # 📌 CS 확장자로 CSV 저장
    df.to_csv(output_cs_path, index=False, encoding='cp949')

    print("✅ 엑셀 및 CS 파일이 저장되었습니다 (5884행 유지, 추가 열 노란색 음영 적용):")
    print(f" - 엑셀 파일: {output_excel_path}")
    print(f" - CS 파일: {output_cs_path}")

except Exception as e:
    print("오류 발생:", e)

✅ 엑셀 및 CS 파일이 저장되었습니다 (5884행 유지, 추가 열 노란색 음영 적용):
 - 엑셀 파일: D:\시설\정지\일일정지\20250212\2025년0212_전사 정지조건리스트_완성.xlsx
 - CS 파일: D:\시설\정지\일일정지\20250212\2025년0212_전사 정지조건리스트_완성.cs


In [12]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from datetime import datetime, timedelta

# 📌 기존 결과 파일 경로
output_excel_path = r'D:\시설\정지\일일정지\20250209\2025년0209_전사 정지조건리스트_완성.xlsx'

# 📌 필터링 후 저장할 새 파일 경로
filtered_output_excel_path = r'D:\시설\정지\일일정지\20250209\2025년0209_월정료없음_제외사유있음.xlsx'
updated_output_excel_path = r'D:\시설\정지\일일정지\20250209\2025년0209_전사 정지조건리스트_최종.xlsx'
unique_contract_output_path = r'D:\시설\정지\일일정지\20250209\2025년0209_전사 정지조건리스트_중복제거.xlsx'

# 📌 기존 결과 파일 불러오기
df = pd.read_excel(output_excel_path, engine='openpyxl')

# 📌 원본 데이터 행 개수 확인
original_count = df.shape[0]
print(f"📌 원본 데이터 행 개수: {original_count}")

# 📌 KTT월정료가 비어 있거나 제외사유가 있는 행 필터링
filtered_df = df[(df['KTT월정료'].isna()) | (df['제외사유'].notna())]

# 📌 필터링된 데이터 별도 저장
filtered_df.to_excel(filtered_output_excel_path, index=False, engine='openpyxl')
filtered_count = filtered_df.shape[0]
print(f"📌 월정료 없음 / 제외사유 있음 데이터 행 개수: {filtered_count}")

# 📌 원본 데이터에서 해당 행 제거 후 업데이트
df = df[(df['KTT월정료'].notna()) & (df['제외사유'].isna())]

# 📌 최종 정리된 데이터 저장
df.to_excel(updated_output_excel_path, index=False, engine='openpyxl')
updated_count = df.shape[0]
print(f"📌 최종 정리된 데이터 행 개수: {updated_count}")

# 📌 데이터 개수 검증
assert original_count == (filtered_count + updated_count), "❌ 오류 발생: 행 개수가 일치하지 않습니다!"

# 📌 날짜 변환 함수
def convert_date(date_str):
    """YYYYMMDD 또는 YYYYMMDDHHMMSS 형식을 YYYY-MM-DD로 변환"""
    try:
        if pd.isna(date_str) or not date_str.strip():
            return None
        date_str = str(date_str).strip()
        if len(date_str) >= 8:
            return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}"
        return None
    except:
        return None

# 📌 날짜 변환 적용
date_columns = ['계약시작일', '계약종료일', '정지시작일자', '정지희망종료일', '계약최초서비스게시일']
for col in date_columns:
    df[col] = df[col].astype(str).apply(convert_date)

today = datetime.today()

# 📌 정지일수 계산 함수
def calculate_freeze_days(start_date, end_date):
    try:
        if not start_date or not end_date:
            return None
        start_date = datetime.strptime(start_date, "%Y-%m-%d")
        end_date = datetime.strptime(end_date, "%Y-%m-%d")
        freeze_days = (end_date - start_date).days + 1
        return freeze_days if freeze_days > 0 else 0
    except:
        return None

df['정지일수'] = df.apply(lambda row: calculate_freeze_days(row['정지시작일자'], row['정지희망종료일']), axis=1)

# 📌 정지일수 구간 분류
def categorize_freeze_days(days):
    if pd.isna(days):
        return None
    elif days <= 89:
        return '89일 이하'
    elif 90 <= days <= 119:
        return '90~119일'
    elif 120 <= days <= 149:
        return '120~149일'
    else:
        return '150일 이상'

df['정지일수 구간'] = df['정지일수'].apply(categorize_freeze_days)

# 📌 종료희망일 초과 여부 추가 (9999-12-31 포함)
def check_exceeding_end_date(start_date, end_date):
    try:
        if not start_date or not end_date:
            return None
        start_date = datetime.strptime(start_date, "%Y-%m-%d")
        end_date = datetime.strptime(end_date, "%Y-%m-%d")

        if end_date.strftime("%Y-%m-%d") == "9999-12-31":
            return '확인대상'
        elif end_date < today:
            return '초과'
        elif (today - start_date).days > 500:
            return '확인대상'
        else:
            return '미초과'
    except:
        return None

df['종료희망일 초과 여부'] = df.apply(
    lambda row: check_exceeding_end_date(row['정지시작일자'], row['정지희망종료일']), axis=1
)

# 📌 KTT월정료 금액 구간 추가
def categorize_fee(amount):
    try:
        amount = float(amount)
        if amount <= 50000:
            return '5만원 이하'
        elif 50000 < amount <= 70000:
            return '5만원 초과 ~ 7만원 이하'
        elif 70000 < amount <= 100000:
            return '7만원 초과 ~ 10만원 이하'
        elif 100000 < amount <= 200000:
            return '10만원 초과 ~ 20만원 이하'
        else:
            return '20만원 초과'
    except:
        return None

df['월정료 구간'] = df['KTT월정료'].apply(categorize_fee)

# 📌 계약번호 기준 중복 제거 및 KTT월정료 합산
df['KTT월정료'] = pd.to_numeric(df['KTT월정료'], errors='coerce').fillna(0)
df_grouped = df.groupby('계약번호', as_index=False).agg(lambda x: x.iloc[0] if x.dtype == 'O' else x.sum())

# 📌 중복 제거된 데이터 저장
df_grouped.to_excel(unique_contract_output_path, index=False, engine='openpyxl')

# 📌 중복 제거 후 행 개수 확인
unique_count = df_grouped.shape[0]
print(f"📌 계약번호 기준 중복 제거 후 데이터 행 개수: {unique_count}")

# 📌 엑셀 파일 열어서 새 열을 노란색 음영(50%) 적용
wb = load_workbook(unique_contract_output_path)
ws = wb.active
yellow_fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")

# 📌 새로 생성된 열(조건 적용된 열)에만 노란색 음영 처리
new_columns = ['정지일수', '정지일수 구간', '종료희망일 초과 여부', '월정료 구간']
for col in new_columns:
    col_index = df_grouped.columns.get_loc(col) + 1  # 엑셀에서는 1부터 시작
    for row_num in range(2, ws.max_row + 1):  # 헤더 제외
        ws.cell(row=row_num, column=col_index).fill = yellow_fill

wb.save(unique_contract_output_path)

print(f"✅ 엑셀 파일 저장 완료! (새 열 노란색 음영 적용)")

📌 원본 데이터 행 개수: 5883
📌 월정료 없음 / 제외사유 있음 데이터 행 개수: 1143
📌 최종 정리된 데이터 행 개수: 4740
📌 계약번호 기준 중복 제거 후 데이터 행 개수: 4729
✅ 엑셀 파일 저장 완료! (새 열 노란색 음영 적용)


## 정지, 계약번호 중복시 월정료 높은것만 존재, 나머지 삭제, 일수, 구간별 적용

# 조건 재정립 20250414

In [19]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

## 정지코드 연습

In [3]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from datetime import datetime, timedelta


# 파일 경로 설정
file_path = r'D:\시설\정지\일일정지\20250225\C150_G0000_00049.csv'  
additional_file_path = r"D:\시설\유지\2025\2025.01월 유지전체변환.cs"
output_excel_path = r'D:\시설\정지\일일정지\20250225\2025년0225_전사 정지조건리스트_완성.xlsx'
output_cs_path = r'D:\시설\정지\일일정지\20250225\2025년0225_전사 정지조건리스트_완성.cs'  

try:
    # 기존 데이터 불러오기 (행 개수 유지)
    df = pd.read_csv(file_path, encoding='cp949', low_memory=False)
    original_row_count = df.shape[0]  


    # ✅ 2. 유지관리 데이터 불러오기 (설치주소 포함)
    additional_df = pd.read_csv(additional_file_path, encoding='cp949', low_memory=False)


    # 필요한 컬럼만 추출
    columns_to_extract = [
        '관리본부명', '관리지사명', '계약번호', '서비스(대)', '서비스(중)', '서비스(소)', '상호',
        '고객구분', '사업용구분', '계약상태(대)', '설치주소', 'KTT월정료', '계약시작일', '계약종료일',
        '영업자명', '정지시작일자', '정지희망종료일', '담당자', '휴대폰', '계약최초서비스게시일',
        '영업구역정보'
    ]

    # 'kt oct 마스터 아이디', '외부고객KEY', '무인매장구분', '무인매장업종구분'
    
    df = df[columns_to_extract]

    # 추가 데이터 불러오기
    additional_df = pd.read_csv(additional_file_path, encoding='cp949', low_memory=False)

    # 중복 제거 (계약번호가 중복되면 첫 번째 값만 사용)
    additional_df = additional_df.drop_duplicates(subset=['계약번호'])

  
    # ✅ 3. 유지파일에서 필요한 컬럼 선택
    columns_to_merge = ['계약번호', '설치주소', '청구번호', '시설구분', '요금구분', '제외사유', '매출구분', '실적채널', '고알프']
    additional_df = additional_df[columns_to_merge].drop_duplicates(subset=['계약번호'])



      # 📌 "관리본부명" 값 변경
      df['관리본부명'] = df['관리본부명'].replace({
        '강원본부': '강북/강원본부',
        '서부본부': '강남/서부본부'
    })

      # ✅ 4. 유지자료에서 매핑 (설치주소 포함)
      df = df.merge(additional_df, on='계약번호', how='left')

      # ✅ 5. 제외사유 값이 존재하면 해당 행 전체 삭제
      df = df[df['제외사유'].isna()]

      # ✅ 6. 계약번호 중복 제거 (KTT월정료 합산 후 가장 높은 금액 유지)
      df['KTT월정료'] = pd.to_numeric(df['KTT월정료'], errors='coerce').fillna(0)
      df = df.sort_values(['계약번호', 'KTT월정료'], ascending=[True, False])
      df = df.groupby('계약번호', as_index=False).agg(lambda x: x.iloc[0] if x.dtype == 'O' else x.sum())


     # 기존 데이터 개수 유지하면서 계약번호 기준 매핑 (NaN 방지)
    for col in additional_columns:
        df[col] = df['계약번호'].map(additional_df.set_index('계약번호')[col]).fillna('')

    # 병합 후 행 개수 검증 (5884 유지)
    assert df.shape[0] == original_row_count, f"오류 발생: 원본 행 개수({original_row_count})와 다름! 현재 행 개수: {df.shape[0]}"

    # 날짜 변환 함수
    def convert_date(date_str):
        """YYYYMMDD 또는 YYYYMMDDHHMMSS 형식을 YYYY-MM-DD로 변환"""
        try:
            if pd.isna(date_str) or not date_str.strip():
                return None
            date_str = str(date_str).strip()
            if len(date_str) >= 8:
                return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}"
            return None
        except:
            return None

    # 날짜 변환 적용
    date_columns = ['계약시작일', '계약종료일', '정지시작일자', '정지희망종료일', '계약최초서비스게시일']
    for col in date_columns:
        df[col] = df[col].astype(str).apply(convert_date)

    today = datetime.today()

    # 정지일수 계산 함수
    def calculate_freeze_days(start_date, end_date):
        try:
            if not start_date or not end_date:
                return None
            start_date = datetime.strptime(start_date, "%Y-%m-%d")
            end_date = datetime.strptime(end_date, "%Y-%m-%d")
            freeze_days = (end_date - start_date).days + 1
            return freeze_days if freeze_days > 0 else 0
        except:
            return None

    df['정지일수'] = df.apply(lambda row: calculate_freeze_days(row['정지시작일자'], row['정지희망종료일']), axis=1)

    # 정지일수 구간 분류
    def categorize_freeze_days(days):
        if pd.isna(days):
            return None
        elif days <= 89:
            return '89일이하'
        elif 90 <= days <= 119:
            return '90~119일'
        elif 120 <= days <= 149:
            return '120~149일'
        else:
            return '150일이상'

    df['정지일수 구간'] = df['정지일수'].apply(categorize_freeze_days)

    # 종료희망일 초과 여부 추가 (9999-12-31 포함)
    def check_exceeding_end_date(start_date, end_date):
        try:
            if not start_date or not end_date:
                return None
            start_date = datetime.strptime(start_date, "%Y-%m-%d")
            end_date = datetime.strptime(end_date, "%Y-%m-%d")

            if end_date.strftime("%Y-%m-%d") == "9999-12-31":
                return '확인대상'
            elif end_date < today:
                return '초과'
            elif (today - start_date).days > 500:
                return '확인대상'
            else:
                return '미초과'
        except:
            return None

    df['종료희망일 초과 여부'] = df.apply(
        lambda row: check_exceeding_end_date(row['정지시작일자'], row['정지희망종료일']), axis=1
    )

    # KTT월정료 금액 구간 추가
    def categorize_fee(amount):
        try:
            amount = float(amount)
            if amount <= 50000:
                return '5만원 이하'
            elif 50000 < amount <= 70000:
                return '5만원 초과 ~ 7만원 이하'
            elif 70000 < amount <= 100000:
                return '7만원 초과 ~ 10만원 이하'
            elif 100000 < amount <= 200000:
                return '10만원 초과 ~ 20만원 이하'
            else:
                return '20만원 초과'
        except:
            return None

    df['월정료 구간'] = df['KTT월정료'].apply(categorize_fee)

    # 새 열을 기존 데이터의 맨 왼쪽으로 이동
    new_columns = additional_columns + ['정지일수', '정지일수 구간', '종료희망일 초과 여부', '월정료 구간'] + columns_to_extract
    df = df[new_columns]

    df.to_excel(output_excel_path, index=False, engine='openpyxl')

    # 엑셀 파일 열어서 새 열을 노란색 음영(50%) 적용
    wb = load_workbook(output_excel_path)
    ws = wb.active
    yellow_fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")

    for col_num in range(1, len(additional_columns) + 5):  # 모든 추가 열에 대해 적용
        for row_num in range(1, ws.max_row + 1):
            ws.cell(row=row_num, column=col_num).fill = yellow_fill

    wb.save(output_excel_path)
    df.to_csv(output_cs_path, index=False, encoding='cp949')

    print(f"엑셀 및 CS 파일 저장 완료 (5884행 유지, 추가 열 노란색 음영 적용)")

except Exception as e:
    print("오류 발생:", e)

IndentationError: unexpected indent (3364359455.py, line 49)

In [ ]:
# ✅ 2. 유지관리 데이터 불러오기 (설치주소 포함)
additional_df = pd.read_csv(additional_file_path, encoding='cp949', low_memory=False)

# ✅ 3. 유지파일에서 필요한 컬럼 선택
columns_to_merge = ['계약번호', '설치주소', '청구번호', '시설구분', '요금구분', '제외사유', '매출구분', '실적채널', '고알프']
additional_df = additional_df[columns_to_merge].drop_duplicates(subset=['계약번호'])

# 📌 "관리본부명" 값 변경
df['관리본부명'] = df['관리본부명'].replace({
    '강원본부': '강북/강원본부',
    '서부본부': '강남/서부본부'
})

# ✅ 4. 유지자료에서 매핑 (설치주소 포함)
df = df.merge(additional_df, on='계약번호', how='left')

# ✅ 5. 제외사유 값이 존재하면 해당 행 전체 삭제
df = df[df['제외사유'].isna()]

# ✅ 6. 계약번호 중복 제거 (KTT월정료 합산 후 가장 높은 금액 유지)
df['KTT월정료'] = pd.to_numeric(df['KTT월정료'], errors='coerce').fillna(0)
df = df.sort_values(['계약번호', 'KTT월정료'], ascending=[True, False])
df = df.groupby('계약번호', as_index=False).agg(lambda x: x.iloc[0] if x.dtype == 'O' else x.sum())

In [1]:
pip install pandas openpyxl tkinter matplotlib chardet pandastable

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tkinter (from versions: none)
ERROR: No matching distribution found for tkinter


In [7]:
pip install pandastable

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/241.5 kB ? eta -:--:--
     --- ------------------------------------ 20.5/241.5 kB ? eta -:--:--
     ------- ----------------------------- 51.2/241.5 kB 871.5 kB/s eta 0:00:01
     -------------------------------------- 241.5/241.5 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/491.3 kB ? eta -:--:--
   --------------------------------------  481.3/491.3 kB 31.4 MB/s eta 0:00:01
   ---------------------------------------- 491.3/491.3 kB 7.8 MB/s eta 0:00:00
  Created wheel for pandastable: filename=pandastable-0.13.1-py3-none-any.whl size=256892 sha256=363df94c873bd4a0993107d251c580a9559afdd586ac1c65505623d358b49672
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\9a\83\4d\b38f2b068b023a5bd9cc5b95c19b6856a5914785561

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


## ✅ **기능 개요**
✔ **사용자가 직접 피벗 테이블의 행/열/값/필터 선택 가능**  
✔ **한글 깨짐 없는 시각화 (Matplotlib, 한글 폰트 적용)**  
✔ **CSV 및 Excel 저장 (인코딩: `949(EUC-KR)`)**  
 **다양한 집계 방식 지원 (SUM, COUNT, MEAN, MAX, MIN)**  

In [11]:
print(data.columns)

Index(['관리본부코드', '관리본부명', '관리지사코드', '관리지사명', '고객번호', '고객명', '계약번호', '유지보수기간',
       '계약자명', '서비스번호',
       ...
       'kt oct 마스터 아이디', '정지희망종료일', '보조MIN번호', '영업지점', '설치지점', '팩토링여부',
       '외부고객KEY', '무인매장구분', '무인매장업종구분', '팩토링상태'],
      dtype='object', length=157)


In [23]:
for col in data.columns: 
    print(col)

관리본부코드
관리본부명
관리지사코드
관리지사명
고객번호
고객명
계약번호
유지보수기간
계약자명
서비스번호
서비스(대)
서비스(중)
서비스(소)
영상전환일
상호
고객구분
사업용구분
주민등록번호
사업자번호
계약상태(대)
계약상태(중)
서비스상태(대)
서비스상태(중)
접속전화번호
설치우편번호
설치주소
견적월정료
KTT월정료
KT월정료
할인율(%)
계약개월수
월정료 면제사유
면제시작일
면제종료일
계약보증금
보증금 면제사유
판매상품비
견적설치공사비
설치공사할부개월
계약공사비(ktt)
계약공사비(kt)
면제공사비
실징수액
계약시작일
계약종료일
영업구분
추천구분
추천경로
영업본부코드
영업본부명
영업지사코드
영업지사명
영업자소속
영업자사번
영업자명
영업자연락처
모집유형
가망고객번호
추천본부코드
추천본부명
추천지사코드
추천지사명
유통망대분류
유통망중분류
유통망소분류
추천자사번
추천자명
추천자유형
정보제공자부서
정보제공자사번
정보제공자명
정보제공자연락처
청구본부코드
청구본부명
청구지사코드
청구지사명
청구자명
합산여부
수납구분
세금계산서발행여부
계산서발행여부
선후납구분
청구번호
청구전화번호
청구우편번호
청구주소
청약일자
청약취소일자
공사희망일
공사예정일
공사완료일
준공완료일
관제개통일
서비스개시일
서비스재개시일
공사의뢰유형
회선방식(대)
회선방식(중)
MUX / SYSTEM_ID
전용회선번호
MIN번호
정지시작일자
해지일자
종목
업태
업종(대)
업종(중)
보험등급
비고
결합구분
결합상품명
결합약정개월수
결합시작일
금융기관코드
면책구분
타사전환
이전고객번호
E-MAIL
담당자
휴대폰
전입신규여부
고객추가정보
계약최초서비스게시일
청약취소사유
계약본부
계약지사
매출본사
출동구역정보
영업구역정보
구역담당지점
구역담당영업사원
선장품
브랜드
보조회선
고객계약유형
A/S운용기간
판매할부개월
월정료일시납
장기요금인상여부
장기요금인상액
장기Upselling여부
장기Upselling매출액
기술구역정보
계약서작성주체
카드사명
GiGAeyes계약Id
월정료
kt oct 마스터 아이디
정지희망종료일
보조

In [25]:
pd.set_option('display.max_columns', None)
print(data.head()) # 데이터 샘플 출력

  관리본부코드    관리본부명 관리지사코드  관리지사명      고객번호        고객명      계약번호  유지보수기간  \
0   O000  전남/전북본부   O510   제주지사  30736522        추*탁  51497532     NaN   
1   O000  전남/전북본부   O420   익산지사  30803933        강*숙  51619691     NaN   
2   J000     서부본부   J050   관악지사  30822965         반*  51651888     NaN   
3   M000  대구/경북본부   M320  서대구지사  30897415  이**뜸길협동조합  51791664     NaN   
4   M000  대구/경북본부   M320  서대구지사  30897415  이**뜸길협동조합  51791664     NaN   

        계약자명     서비스번호 서비스(대) 서비스(중)                   서비스(소)  영상전환일  \
0        추*탁  62004450  기본서비스  텔레캅아이  ktt GiGAeyes view+(DVR)    NaN   
1        강*숙  62263339  기본서비스  텔레캅아이          GiGAeyes i-slim    NaN   
2         반*  62333727  기본서비스  텔레캅아이    GiGAeyes i-slim(2210)    NaN   
3  이**뜸길협동조합  62625535  기본서비스  텔레캅아이     GiGAeyes guard+(DVR)    NaN   
4  이**뜸길협동조합  62625536  기본서비스  텔레캅아이     GiGAeyes guard+(DVR)    NaN   

                          상호       고객구분 사업용구분          주민등록번호         사업자번호  \
0                     추형탁님주택         개인  비사

In [ ]:
!pip install --upgrade gspread gspread_dataframe oauth2client

## 정지 개선작업2_20250414

In [ ]:
# ✅ Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# ✅ 인증 키 경로 설정
json_keyfile_path = "/content/drive/MyDrive/Key/credentials.json"

# ✅ 인증 및 gspread 연결
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_keyfile_path, scope)
gc = gspread.authorize(credentials)

# ✅ 스프레드시트 열기
spreadsheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/13hT-Ea5-P5Vi-37d1dZp2ev_Mi3QUQ5Rh9XT33sqxhA/edit")
worksheet = spreadsheet.worksheet("DB_04.03")

# ✅ 데이터 가져오기
sheet_df = get_as_dataframe(worksheet).dropna(subset=['계약번호'])
mapping_df = sheet_df[['계약번호', 'L/i형구분', '체납구분1', '체납구분2']].dropna()

# ✅ 기존 df에 병합
df['L/i형구분'] = df['계약번호'].map(mapping_df.set_index('계약번호')['L/i형구분']).fillna('')
df['체납구분1'] = df['계약번호'].map(mapping_df.set_index('계약번호')['체납구분1']).fillna('')
df['체납구분2'] = df['계약번호'].map(mapping_df.set_index('계약번호')['체납구분2']).fillna('')

In [11]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from datetime import datetime
import calendar
import chardet

# ✅ 인코딩 감지 함수
def detect_encoding(file_path, max_bytes=100000):
    with open(file_path, 'rb') as f:
        sample = f.read(max_bytes)
        result = chardet.detect(sample)
        encoding = result['encoding']
        if encoding and 'euc' in encoding.lower():
            encoding = 'cp949'
        return encoding or 'cp949'

# ✅ 파일 경로 설정
file_path = r'D:\시설\정지\일일정지\20250425\C155_03104_00045.csv'
additional_file_path = r"D:\시설\유지\2025\3월\3월마감 조건추출 선택컬럼_2403.csv"
output_excel_path = r'D:\시설\정지\일일정지\20250425\2025년0425_전사 정지조건리스트_완성_서비스번호.xlsx'
output_csv_path = r'D:\시설\정지\일일정지\20250425\2025년0425_전사 정지조건리스트_완성_서비스번호.csv'

try:
    # ✅ 데이터 불러오기
    df = pd.read_csv(file_path, encoding=detect_encoding(file_path), low_memory=False)
    additional_df = pd.read_csv(additional_file_path, encoding=detect_encoding(additional_file_path), low_memory=False)

    # ✅ 주요 컬럼 필터링
    columns_to_extract = [
        '관리본부명', '관리지사명', '고객번호', '계약번호', '서비스번호', '서비스(대)', '서비스(중)', '서비스(소)',
        '상호', '고객구분', '사업용구분', '계약상태(대)', '설치주소', 'KTT월정료(조정)', 'KTT월정료',
        '계약시작일', '계약종료일', '영업자명', '정지시작일자', '정지희망종료일', '계약최초서비스게시일', '영업구역정보'
    ]
    df = df[[col for col in columns_to_extract if col in df.columns]]
    df = df[(df['서비스(대)'] == '기본서비스') & (df['사업용구분'] != '사업용')]

    # ✅ 본부명 정제
    df['관리본부명'] = df['관리본부명'].replace({
        '강원본부': '강북/강원본부',
        '서부본부': '강남/서부본부'
    })

    # ✅ 추가 컬럼 병합
    additional_columns = ['시설구분', '요금구분', '제외사유', '매출구분', '실적채널', '고알프', '설치주소']
    additional_df = additional_df.drop_duplicates(subset=['계약번호'])
    additional_df = additional_df[['계약번호'] + [col for col in additional_columns if col in additional_df.columns]]

    for col in additional_columns:
        df[col] = df['계약번호'].map(additional_df.set_index('계약번호')[col]).fillna('')

    # ✅ 제외사유 제거
    if '제외사유' in df.columns:
        df['제외사유'] = df['제외사유'].astype(str).str.strip()
        df = df[df['제외사유'] == '']

    # ✅ KTT월정료 통일: 'KTT월정료(조정)' → 'KTT월정료' 우선 적용
    if 'KTT월정료(조정)' in df.columns:
        df['KTT월정료'] = df['KTT월정료(조정)']
    elif 'KTT월정료' not in df.columns:
        df['KTT월정료'] = ''  # 없는 경우 빈 컬럼 생성

    # ✅ 날짜 포맷 정제 함수
    def convert_date(date_str):
        try:
            if pd.isna(date_str) or not str(date_str).strip():
                return None
            date_str = str(date_str).strip()
            if len(date_str) >= 8:
                return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}"
            return None
        except:
            return None

    for col in ['계약시작일', '계약종료일', '정지시작일자', '정지희망종료일', '계약최초서비스게시일']:
        if col in df.columns:
            df[col] = df[col].astype(str).apply(convert_date)

    # ✅ 종료희망일 보정
    today = datetime.today()
    last_day = datetime(today.year, today.month, calendar.monthrange(today.year, today.month)[1])
    last_day_str = last_day.strftime("%Y-%m-%d")
    df['정지희망종료일'] = df['정지희망종료일'].apply(
        lambda x: last_day_str if not x or x == '9999-12-31' else x
    )

    # ✅ 종료희망일 초과 여부
    def check_exceeding(row):
        try:
            end_date = datetime.strptime(row['정지희망종료일'], '%Y-%m-%d')
            return '초과' if end_date <= last_day else '미초과'
        except:
            return None

    df['종료희망일 초과 여부'] = df.apply(check_exceeding, axis=1)

    # ✅ 정지일수 계산
    def calculate_freeze_days(start, end):
        try:
            if not start or not end:
                return None
            start = datetime.strptime(start, "%Y-%m-%d")
            end = datetime.strptime(end, "%Y-%m-%d")
            return max((end - start).days + 1, 0)
        except:
            return None

    df['정지일수'] = df.apply(lambda row: calculate_freeze_days(row['정지시작일자'], row['정지희망종료일']), axis=1)

    # ✅ 정지일수 구간
    def categorize_freeze_days(days):
        if pd.isna(days): return None
        elif days <= 89: return '89일이하'
        elif days <= 119: return '90~119일'
        elif days <= 149: return '120~149일'
        else: return '150일이상'

    df['정지일수 구간'] = df['정지일수'].apply(categorize_freeze_days)

    # ✅ 월정료 구간 분류
    def categorize_fee(val):
        try:
            amount = float(str(val).replace(',', ''))
            if amount <= 50000:
                return '5만 이하'
            elif amount <= 70000:
                return '5만 ~ 7만 이하'
            elif amount <= 100000:
                return '7만 ~ 10만 이하'
            elif amount <= 200000:
                return '10만 ~ 20만 이하'
            elif amount <= 500000:
                return '20만 ~ 50만 이하'
            else:
                return '50만 초과'
        except:
            return None

    df['월정료 구간'] = df['KTT월정료'].apply(categorize_fee)

    # ✅ 컬럼 순서 정리
    final_columns = additional_columns + ['정지일수', '정지일수 구간', '종료희망일 초과 여부', '월정료 구간'] + columns_to_extract
    df = df[[col for col in final_columns if col in df.columns]]

    # ✅ 엑셀 저장
    df.to_excel(output_excel_path, index=False, engine='openpyxl')

    # ✅ 노란색 강조 처리
    wb = load_workbook(output_excel_path)
    ws = wb.active
    yellow_fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")
    for col in range(1, len(additional_columns) + 5):
        for row in range(2, ws.max_row + 1):
            ws.cell(row=row, column=col).fill = yellow_fill
    wb.save(output_excel_path)

    # ✅ CSV 저장 (한글 호환용)
    df.to_csv(output_csv_path, index=False, encoding='cp949')

    print(f"✅ 최종 저장 완료! (행 수: {df.shape[0]})")

except Exception as e:
    print("❌ 오류 발생:", e)

❌ 오류 발생: 'cp949' codec can't decode byte 0xab in position 12: illegal multibyte sequence


In [ ]:
##중복제거0425

In [33]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from datetime import datetime
import calendar
import chardet
import os

# ✅ 인코딩 감지 함수
def detect_encoding(file_path, max_bytes=100000):
    with open(file_path, 'rb') as f:
        sample = f.read(max_bytes)
        result = chardet.detect(sample)
        encoding = result['encoding']
        if encoding and 'euc' in encoding.lower():
            encoding = 'cp949'
        return encoding or 'cp949'

# ✅ 경로 설정
file_path = r'D:\시설\정지\일일정지\20250425\C155_03104_00045.csv'
additional_file_path = r"D:\시설\유지\2025\3월\3월마감 조건추출 선택컬럼_2403.csv"
output_excel_path = r'D:\시설\정지\일일정지\20250425\2025년0425_전사 정지조건리스트_완성_중복제거0425.xlsx'
output_csv_path = r'D:\시설\정지\일일정지\20250425\2025년0425_전사 정지조건리스트_완성_중복제거0425.csv'

try:
    # ✅ CSV 로드
    df = pd.read_csv(file_path, encoding=detect_encoding(file_path), low_memory=False)
    additional_df = pd.read_csv(additional_file_path, encoding=detect_encoding(additional_file_path), low_memory=False)

    # ✅ 주요 컬럼 필터링
    columns_to_extract = [
        '관리본부명', '관리지사명', '고객번호', '계약번호', '서비스번호', '서비스(대)', '서비스(중)', '서비스(소)',
        '상호', '고객구분', '사업용구분', '계약상태(대)', '설치주소', 'KTT월정료(조정)', 'KTT월정료',
        '계약시작일', '계약종료일', '영업자명', '정지시작일자', '정지희망종료일', '계약최초서비스게시일', '영업구역정보'
    ]
    df = df[[col for col in columns_to_extract if col in df.columns]]
    df = df[(df['서비스(대)'] == '기본서비스') & (df['사업용구분'] != '사업용')]

    # ✅ 본부명 정제
    df['관리본부명'] = df['관리본부명'].replace({
        '강원본부': '강북/강원본부',
        '서부본부': '강남/서부본부'
    })

    # ✅ 추가 컬럼 병합
    additional_columns = ['시설구분', '요금구분', '제외사유', '매출구분', '실적채널', '고알프', '설치주소']
    additional_df = additional_df.drop_duplicates(subset=['계약번호'])
    additional_df = additional_df[['계약번호'] + [col for col in additional_columns if col in additional_df.columns]]

    for col in additional_columns:
        df[col] = df['계약번호'].map(additional_df.set_index('계약번호')[col]).fillna('')

    # ✅ 제외사유 제거
    if '제외사유' in df.columns:
        df['제외사유'] = df['제외사유'].astype(str).str.strip()
        df = df[df['제외사유'] == '']

    # ✅ KTT월정료 통일
    if 'KTT월정료(조정)' in df.columns:
        df['KTT월정료'] = df['KTT월정료(조정)']
    elif 'KTT월정료' not in df.columns:
        df['KTT월정료'] = ''

    df['KTT월정료'] = pd.to_numeric(df['KTT월정료'], errors='coerce').fillna(0)

    # ✅ 날짜 포맷 정제 함수
    def convert_date(date_str):
        try:
            if pd.isna(date_str) or not str(date_str).strip():
                return None
            date_str = str(date_str).strip()
            if len(date_str) >= 8:
                return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}"
            return None
        except:
            return None

    for col in ['계약시작일', '계약종료일', '정지시작일자', '정지희망종료일', '계약최초서비스게시일']:
        if col in df.columns:
            df[col] = df[col].astype(str).apply(convert_date)

    # ✅ 종료희망일 보정
    today = datetime.today()
    last_day = datetime(today.year, today.month, calendar.monthrange(today.year, today.month)[1])
    last_day_str = last_day.strftime("%Y-%m-%d")
    df['정지희망종료일'] = df['정지희망종료일'].apply(
        lambda x: last_day_str if not x or x == '9999-12-31' else x
    )

    # ✅ 종료희망일 초과 여부
    def check_exceeding(row):
        try:
            end_date = datetime.strptime(row['정지희망종료일'], '%Y-%m-%d')
            return '초과' if end_date <= last_day else '미초과'
        except:
            return None

    df['종료희망일 초과 여부'] = df.apply(check_exceeding, axis=1)

    # ✅ 정지일수 계산
    def calculate_freeze_days(start, end):
        try:
            if not start or not end:
                return None
            start = datetime.strptime(start, "%Y-%m-%d")
            end = datetime.strptime(end, "%Y-%m-%d")
            return max((end - start).days + 1, 0)
        except:
            return None

    df['정지일수'] = df.apply(lambda row: calculate_freeze_days(row['정지시작일자'], row['정지희망종료일']), axis=1)

    # ✅ 정지일수 구간
    def categorize_freeze_days(days):
        if pd.isna(days): return None
        elif days <= 89: return '89일이하'
        elif days <= 119: return '90~119일'
        elif days <= 149: return '120~149일'
        else: return '150일이상'

    df['정지일수 구간'] = df['정지일수'].apply(categorize_freeze_days)

    # ✅ 월정료 구간 분류
    def categorize_fee(val):
        try:
            amount = float(str(val).replace(',', ''))
            if amount <= 50000:
                return '5만원 이하'
            elif amount <= 70000:
                return '5만원 초과 ~ 7만원 이하'
            elif amount <= 100000:
                return '7만원 초과 ~ 10만원 이하'
            elif amount <= 200000:
                return '10만원 초과 ~ 20만원 이하'
            else:
                return '20만원 초과'
        except:
            return None

    df['월정료 구간'] = df['KTT월정료'].apply(categorize_fee)

    # ✅ 계약번호 기준 중복 제거 + KTT월정료 합산
    df = df.groupby('계약번호', as_index=False).agg({
        **{col: 'first' for col in df.columns if col not in ['계약번호', 'KTT월정료']},
        'KTT월정료': 'sum'
    })

    # ✅ 컬럼 정렬
    final_columns = additional_columns + ['정지일수', '정지일수 구간', '종료희망일 초과 여부', '월정료 구간'] + columns_to_extract
    df = df[[col for col in final_columns if col in df.columns]]

    # ✅ 엑셀 저장
    df.to_excel(output_excel_path, index=False, engine='openpyxl')

    # ✅ 강조 색상 적용 (노란색)
    wb = load_workbook(output_excel_path)
    ws = wb.active
    yellow_fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")
    for col in range(1, len(additional_columns) + 5):
        for row in range(2, ws.max_row + 1):
            ws.cell(row=row, column=col).fill = yellow_fill
    wb.save(output_excel_path)

    # ✅ CSV 저장 (한글용)
    df.to_csv(output_csv_path, index=False, encoding='cp949')

    print(f"\n✅ 최종 저장 완료! 행 수: {df.shape[0]}")
    print(f"📄 엑셀 저장 경로: {output_excel_path}")
    print(f"📄 CSV 저장 경로: {output_csv_path}")

except Exception as e:
    print("❌ 오류 발생:", e)


✅ 최종 저장 완료! 행 수: 5170
📄 엑셀 저장 경로: D:\시설\정지\일일정지\20250425\2025년0425_전사 정지조건리스트_완성_중복제거0425.xlsx
📄 CSV 저장 경로: D:\시설\정지\일일정지\20250425\2025년0425_전사 정지조건리스트_완성_중복제거0425.csv


In [41]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from datetime import datetime
import calendar
import chardet
import os

# ✅ 인코딩 감지 함수
def detect_encoding(file_path, max_bytes=100000):
    with open(file_path, 'rb') as f:
        sample = f.read(max_bytes)
        result = chardet.detect(sample)
        encoding = result['encoding']
        if encoding and 'euc' in encoding.lower():
            encoding = 'cp949'
        return encoding or 'cp949'

# ✅ 경로 설정
file_path = r'D:\시설\정지\일일정지\20250430\C150_G0000_00017.csv'
additional_file_path = r"D:\시설\유지\2025\3월\3월마감 조건추출 선택컬럼_2403.csv"
output_excel_path = r'D:\시설\정지\일일정지\20250430\2025년0430_전사 정지조건리스트_완성_중복제거0430-2.xlsx'
output_csv_path = r'D:\시설\정지\일일정지\20250430\2025년0430_전사 정지조건리스트_완성_중복제거0430-2.csv'


try:
    # ✅ 데이터 로드
    df = pd.read_csv(file_path, encoding=detect_encoding(file_path), low_memory=False)
    additional_df = pd.read_csv(additional_file_path, encoding=detect_encoding(additional_file_path), low_memory=False)

    # ✅ 필수 컬럼 정리
    columns_to_extract = [
        '관리본부명', '관리지사명', '고객번호', '계약번호', '서비스번호', '서비스(대)', '서비스(중)', '서비스(소)',
        '상호', '고객구분', '사업용구분', '계약상태(대)', '설치주소', 'KTT월정료(조정)', 'KTT월정료',
        '계약시작일', '계약종료일', '영업자명', '정지시작일자', '정지희망종료일', '계약최초서비스게시일', '영업구역정보'
    ]
    df = df[[col for col in columns_to_extract if col in df.columns]]
    df = df[(df['서비스(대)'] == '기본서비스') & (df['사업용구분'] != '사업용')]

    # ✅ 본부명 통일
    df['관리본부명'] = df['관리본부명'].replace({
        '강원본부': '강북/강원본부',
        '서부본부': '강남/서부본부'
    })

    # ✅ 추가 정보 병합
    additional_columns = ['시설구분', '요금구분', '제외사유', '매출구분', '실적채널', '고알프', '설치주소']
    additional_df = additional_df.drop_duplicates(subset=['계약번호'])
    additional_df = additional_df[['계약번호'] + [col for col in additional_columns if col in additional_df.columns]]

    for col in additional_columns:
        df[col] = df['계약번호'].map(additional_df.set_index('계약번호')[col]).fillna('')

    # ✅ 제외사유 제거
    if '제외사유' in df.columns:
        df['제외사유'] = df['제외사유'].astype(str).str.strip()
        df = df[df['제외사유'] == '']

    # ✅ KTT월정료 통일 및 숫자화
    if 'KTT월정료(조정)' in df.columns:
        df['KTT월정료'] = df['KTT월정료(조정)']
    elif 'KTT월정료' not in df.columns:
        df['KTT월정료'] = ''

    df['KTT월정료'] = pd.to_numeric(df['KTT월정료'], errors='coerce').fillna(0)

    # ✅ 날짜 포맷 정제
    def convert_date(date_str):
        try:
            if pd.isna(date_str) or not str(date_str).strip():
                return None
            date_str = str(date_str).strip()
            if len(date_str) >= 8:
                return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}"
            return None
        except:
            return None

    for col in ['계약시작일', '계약종료일', '정지시작일자', '정지희망종료일', '계약최초서비스게시일']:
        if col in df.columns:
            df[col] = df[col].astype(str).apply(convert_date)

    # ✅ 종료희망일 보정
    today = datetime.today()
    last_day = datetime(today.year, today.month, calendar.monthrange(today.year, today.month)[1])
    last_day_str = last_day.strftime("%Y-%m-%d")
    df['정지희망종료일'] = df['정지희망종료일'].apply(
        lambda x: last_day_str if not x or x == '9999-12-31' else x
    )

    # ✅ 종료희망일 초과 여부
    def check_exceeding(row):
        try:
            end_date = datetime.strptime(row['정지희망종료일'], '%Y-%m-%d')
            return '초과' if end_date <= last_day else '미초과'
        except:
            return None

    df['종료희망일 초과 여부'] = df.apply(check_exceeding, axis=1)

    # ✅ 정지일수 계산
    def calculate_freeze_days(start, end):
        try:
            if not start or not end:
                return None
            start = datetime.strptime(start, "%Y-%m-%d")
            end = datetime.strptime(end, "%Y-%m-%d")
            return max((end - start).days + 1, 0)
        except:
            return None

    df['정지일수'] = df.apply(lambda row: calculate_freeze_days(row['정지시작일자'], row['정지희망종료일']), axis=1)

    # ✅ 정지일수 구간
    def categorize_freeze_days(days):
        if pd.isna(days): return None
        elif days <= 89: return '89일이하'
        elif days <= 119: return '90~119일'
        elif days <= 149: return '120~149일'
        else: return '150일이상'

    df['정지일수 구간'] = df['정지일수'].apply(categorize_freeze_days)

    # ✅ 계약번호 기준 중복 제거 + KTT월정료 합산
    df = df.groupby('계약번호', as_index=False).agg({
        **{col: 'first' for col in df.columns if col not in ['계약번호', 'KTT월정료']},
        'KTT월정료': 'sum'
    })

    # ✅ 월정료 구간 재계산
    def categorize_fee(val):
        try:
            if pd.isna(val): return None
            amount = float(str(val).replace(',', ''))
            if amount <= 50000:
                return '5만 이하'
            elif amount <= 70000:
                return '5만 ~ 7만 이하'
            elif amount <= 100000:
                return '7만 ~ 10만 이하'
            elif amount <= 200000:
                return '10만 초과 ~ 20만 이하'
            elif amount <= 500000:
                return '20만 ~ 50만 이하'
            else:
                return '50만 초과'
        except:
            return None

    df['월정료 구간'] = df['KTT월정료'].apply(categorize_fee)

    # ✅ 컬럼 순서 정리
    final_columns = additional_columns + ['정지일수', '정지일수 구간', '종료희망일 초과 여부', '월정료 구간'] + columns_to_extract
    df = df[[col for col in final_columns if col in df.columns]]

    # ✅ 엑셀 저장
    df.to_excel(output_excel_path, index=False, engine='openpyxl')

    # ✅ 강조 색상 (노란색)
    wb = load_workbook(output_excel_path)
    ws = wb.active
    yellow_fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")
    for col in range(1, len(additional_columns) + 5):
        for row in range(2, ws.max_row + 1):
            ws.cell(row=row, column=col).fill = yellow_fill
    wb.save(output_excel_path)

    # ✅ CSV 저장 (한글용)
    df.to_csv(output_csv_path, index=False, encoding='cp949')

    print(f"\n✅ 최종 저장 완료! 행 수: {df.shape[0]}")
    print(f"📄 엑셀 저장 경로: {output_excel_path}")
    print(f"📄 CSV 저장 경로: {output_csv_path}")

except Exception as e:
    print("❌ 오류 발생:", e)


✅ 최종 저장 완료! 행 수: 5112
📄 엑셀 저장 경로: D:\시설\정지\일일정지\20250430\2025년0430_전사 정지조건리스트_완성_중복제거0430-2.xlsx
📄 CSV 저장 경로: D:\시설\정지\일일정지\20250430\2025년0430_전사 정지조건리스트_완성_중복제거0430-2.csv
